### Ensure no other containers are running for dev container, if they are stop and remove them


In [1]:
import os 
print(os.getcwd())
if os.path.exists("notebooks/5080_gpu"):
    os.chdir("notebooks/5080_gpu")
    print(os.getcwd())
else:
    print("notebooks/5080_gpu does not exist")
    print(os.getcwd())
    print("Please run this script from the root of the repository")
    print("e.g. python environment_uv_docker_jax_pytorch_mlflow_portfriendly_5080_allenabled_efficient.ipynb")
    print("or cd to the root of the repository and run this script")
    print("e.g. cd .. && python notebooks/5080_gpu/environment_uv_docker_jax_pytorch_mlflow_portfriendly_5080_allenabled_efficient.ipynb")
    exit(1)

c:\docker_projects\cancerbayes_irisrf_ex_fastapi_react\notebooks\5080_gpu
notebooks/5080_gpu does not exist
c:\docker_projects\cancerbayes_irisrf_ex_fastapi_react\notebooks\5080_gpu
Please run this script from the root of the repository
e.g. python environment_uv_docker_jax_pytorch_mlflow_portfriendly_5080_allenabled_efficient.ipynb
or cd to the root of the repository and run this script
e.g. cd .. && python notebooks/5080_gpu/environment_uv_docker_jax_pytorch_mlflow_portfriendly_5080_allenabled_efficient.ipynb


In [2]:
%%writefile ../../.gitattributes
# Enforce LF for shell scripts & Docker/compose to avoid bash/sh\r errors
*.sh     text eol=lf
*.bash   text eol=lf
Dockerfile text eol=lf
*.dockerfile text eol=lf
docker-compose*.yml text eol=lf
*.env    text eol=lf



Overwriting ../../.gitattributes


In [3]:
%%writefile ../../.dockerignore
# VCS/editor
.git
.gitignore
.vscode
.idea
.DS_Store
Thumbs.db

# Python
**/__pycache__/
**/*.py[cod]
**/*.pyo
*.log

# Virtual envs
.venv
venv
env

# Data / artifacts
data/
mlruns/
mlflow_db/
**/.ipynb_checkpoints/

# Big notebooks outputs
notebooks/**/*.ipynb_checkpoints

# OS / misc
*.code-workspace

# We still need the .devcontainer directory for the Dockerfile, scripts, etc.




Overwriting ../../.dockerignore


In [1]:
%%writefile ../../.devcontainer/.env.template 
ENV_NAME=cancer_bayes_iris_env
CUDA_TAG=12.8.0
DOCKER_BUILDKIT=1

# Host ports
HOST_JUPYTER_PORT=8896
HOST_TENSORBOARD_PORT=6006
HOST_EXPLAINER_PORT=8056
HOST_STREAMLIT_PORT=8506
HOST_MLFLOW_PORT=5006

# Python & JAX
PYTHON_VER=3.10

# Leave blank to autodetect; set to 'gpu' or 'cpu' only if you must force it.
JAX_PLATFORM_NAME=

XLA_PYTHON_CLIENT_PREALLOCATE=true
XLA_PYTHON_CLIENT_ALLOCATOR=platform
XLA_PYTHON_CLIENT_MEM_FRACTION=0.95
XLA_FLAGS=--xla_force_host_platform_device_count=1
JAX_DISABLE_JIT=false
JAX_ENABLE_X64=false
TF_FORCE_GPU_ALLOW_GROWTH=false
JAX_PREALLOCATION_SIZE_LIMIT_BYTES=8589934592

# Runtime GPU targeting & Jupyter convenience
JUPYTER_TOKEN=jupyter

# Keep uv from targeting /workspace/.venv (a bind mount)
UV_PROJECT_ENVIRONMENT=/app/.venv


# NOTE: Torch CUDA wheel is auto-selected from CUDA_TAG at startup:
#   12.1  -> cu121
#   12.4  -> cu124
#   12.6+ -> cu126 (works for 12.6 / 12.8 / 12.9)


# ── Kaggle authentication (choose ONE approach) ─────────────────────────
# Option A: env vars (fastest for CI/containers)
KAGGLE_USERNAME=geoffhadfield 
KAGGLE_KEY=your_api_token



Overwriting ../../.devcontainer/.env.template


In [5]:
%%writefile ../../.devcontainer/devcontainer.json
{
  "name": "cancer_bayes_iris_env_uv",
  "dockerComposeFile": ["docker-compose.yml"],
  "service": "datascience",
  "workspaceFolder": "/workspace",
  "shutdownAction": "stopCompose",

  "customizations": {
    "vscode": {
      "extensions": [
        "ms-python.python",
        "ms-python.vscode-pylance",
        "ms-toolsai.jupyter",
        "ms-toolsai.jupyter-renderers",
        "nvidia.nsight-vscode-edition"
      ],
      "settings": {
        "telemetry.telemetryLevel": "off",
        "python.telemetry.enabled": false,
        "jupyter.telemetry.enabled": false,
        "jupyter.experiments.enabled": false,
        "update.mode": "manual",
        "extensions.autoUpdate": false,
        "extensions.autoCheckUpdates": false,
        "remote.extensionKind": {
          "ms-python.python": ["ui"],
          "ms-python.vscode-pylance": ["ui"],
          "ms-toolsai.jupyter": ["ui"],
          "ms-toolsai.jupyter-renderers": ["ui"],
          "nvidia.nsight-vscode-edition": ["ui"]
        },
        "python.defaultInterpreterPath": "/app/.venv/bin/python",
        "jupyter.interactiveWindow.textEditor.executeSelection": true,
        "jupyter.widgetScriptSources": ["jsdelivr.com", "unpkg.com"]
      }
    }
  },

  "containerEnv": {
    "UV_PROJECT_ENVIRONMENT": "/app/.venv",
    "NVIDIA_VISIBLE_DEVICES": "all",
    "NVIDIA_DRIVER_CAPABILITIES": "compute,utility",
    "CUDA_STACK": "pip-cuda",
    "LD_PRELOAD": "/usr/lib/x86_64-linux-gnu/libjemalloc.so.2",
    "MALLOC_ARENA_MAX": "1",
    "MALLOC_TCACHE_MAX": "0",
    "PYTORCH_NO_CUDA_MEMORY_CACHING": "1",
    "XLA_PYTHON_CLIENT_PREALLOCATE": "false",
    "XLA_PYTHON_CLIENT_MEM_FRACTION": "0.25",
    "XLA_PYTHON_CLIENT_ALLOCATOR": "platform",
    "PYTORCH_CUDA_ALLOC_CONF": "max_split_size_mb:512,expandable_segments:True"
  },

  "remoteEnv": {
    "JUPYTER_ENABLE_LAB": "true",
    "PATH": "${containerEnv:PATH}:/usr/local/cuda/bin"
  },

  "postCreateCommand": "/bin/sh -lc 'set -e; find .devcontainer -maxdepth 1 -name \"*.sh\" -exec sed -i \"s/\\r$//\" {} \\;; chmod +x .devcontainer/*.sh; bash .devcontainer/opening.sh'",
  "postStartCommand": "/bin/sh -lc 'set -e; find .devcontainer -maxdepth 1 -name \"*.sh\" -exec sed -i \"s/\\r$//\" {} \\;; chmod +x .devcontainer/*.sh; bash .devcontainer/opening.sh --verify-only || true'"
}




Overwriting ../../.devcontainer/devcontainer.json


In [6]:
%%writefile ../../.devcontainer/Dockerfile
# .devcontainer/Dockerfile - Optimized for faster builds
ARG CUDA_TAG=12.8.0
# Use NVIDIA's runtime image with pre-installed CUDA libraries (saves 1+ GB downloads)
FROM nvidia/cuda:${CUDA_TAG}-runtime-ubuntu22.04 as runtime-base
FROM nvidia/cuda:${CUDA_TAG}-devel-ubuntu22.04

ARG PYTHON_VER=3.10
ARG ENV_NAME=cancer_bayes_iris_env
ENV DEBIAN_FRONTEND=noninteractive

# Single RUN layer for system packages with optimized cache mounts
RUN --mount=type=cache,target=/var/cache/apt,sharing=locked \
    --mount=type=cache,target=/var/lib/apt,sharing=locked \
    apt-get update && apt-get install -y --no-install-recommends \
        # Essential tools
        bash curl ca-certificates git procps htop util-linux \
        python3 python3-venv python3-pip python3-dev \
        # Build tools (minimal set)
        build-essential cmake pkg-config \
        # Memory management 
        libjemalloc2 libjemalloc-dev \
        # Network tools for diagnostics
        iproute2 net-tools lsof \
    && apt-get clean && rm -rf /var/lib/apt/lists/*

# Node.js in single layer with cache mount
RUN --mount=type=cache,target=/var/cache/apt,sharing=locked \
    --mount=type=cache,target=/var/lib/apt,sharing=locked \
    curl -fsSL https://deb.nodesource.com/setup_18.x | bash - \
    && apt-get install -y nodejs \
    && rm -rf /var/lib/apt/lists/*

# Install uv package manager (pinned version for reproducibility)
COPY --from=ghcr.io/astral-sh/uv:0.7.12 /uv /uvx /bin/

WORKDIR /app

# Create virtual environment early
RUN uv venv .venv --python "${PYTHON_VER}" --prompt "${ENV_NAME}"

ENV VIRTUAL_ENV=/app/.venv \
    PATH="/app/.venv/bin:${PATH}" \
    UV_PROJECT_ENVIRONMENT=/app/.venv

# Essential memory management (simplified from original complex setup)
ENV LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libjemalloc.so.2 \
    MALLOC_ARENA_MAX=1 \
    MALLOC_TCACHE_MAX=0 \
    PYTORCH_NO_CUDA_MEMORY_CACHING=1

# GPU framework environment (JAX & PyTorch only, TensorFlow removed)
ENV XLA_PYTHON_CLIENT_PREALLOCATE=false \
    XLA_PYTHON_CLIENT_MEM_FRACTION=0.25 \
    XLA_PYTHON_CLIENT_ALLOCATOR=platform \
    PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512,expandable_segments:True \
    JAX_PREALLOCATION_SIZE_LIMIT_BYTES=8589934592

# Install core Python packages first (better caching)
RUN --mount=type=cache,target=/root/.cache/uv \
    uv pip install --no-cache-dir jupyterlab ipykernel

# CUDA symlink setup (simplified)
RUN ln -sf $(find /usr/local -name "cuda-*" -type d | head -n1) /usr/local/cuda || \
    ln -sf /usr/local/cuda /usr/local/cuda

# Copy project files for dependency installation (from repo root)
COPY pyproject.toml /workspace/
COPY uv.lock* /workspace/

# Install project dependencies with better error handling
RUN --mount=type=cache,target=/root/.cache/uv \
    cd /workspace && \
    (uv sync --frozen --no-dev || (echo "🔁 Lock out-of-date; refreshing…" && uv sync --no-dev && uv lock))

# Essential CUDA libraries path
ENV LD_LIBRARY_PATH="/app/.venv/lib:/usr/local/cuda/lib64:${LD_LIBRARY_PATH}"

# Workspace setup
WORKDIR /workspace
RUN echo 'cd /workspace' > /etc/profile.d/99-workspace-cd.sh && \
    mkdir -p /root/.ipython/profile_default/startup && \
    printf "import os, sys\nos.chdir('/workspace')\nsys.path.append('/workspace')\n" \
      > /root/.ipython/profile_default/startup/00-cd-workspace.py && \
    echo '. /app/.venv/bin/activate' > /etc/profile.d/10-uv-activate.sh

CMD ["bash"]



Overwriting ../../.devcontainer/Dockerfile


In [7]:
%%writefile ../../.devcontainer/verify_env.py
#!/usr/bin/env python3
import sys, importlib, os, textwrap

CRIT = []
WARN = []

def _have(mod: str) -> bool:
    try:
        importlib.import_module(mod); return True
    except Exception:
        return False

def _msg_box(title: str, body: str) -> None:
    line = "=" * 80
    print(f"\n{line}\n{title}\n{line}\n{body}\n")

def _probe_jax() -> None:
    """
    Probe JAX availability and devices; detect duplicate PJRT CUDA packages and
    mixed CUDA stacks (system + pip nvidia-*), which commonly trigger allocator
    double-frees inside XLA/PJRT.
    """
    import importlib.util as u
    import importlib, os, textwrap, subprocess

    jp = os.environ.get("JAX_PLATFORM_NAME", "<unset>")
    print(f"   JAX_PLATFORM_NAME: {jp}")

    try:
        jax = importlib.import_module("jax")
    except Exception as e:
        WARN.append(f"jax not importable: {e!r}")
        _msg_box(
            "Action: JAX not importable",
            "• Install JAX into /app/.venv:\n"
            "    uv pip install 'jax[cuda12-local]' -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html\n"
            "• Avoid bare 'pip'; prefer 'uv pip' with UV_PROJECT_ENVIRONMENT=/app/.venv ."
        )
        return

    try:
        jaxlib = importlib.import_module("jaxlib")
        print(f"   jax: {getattr(jax,'__version__','?')} @ {getattr(jax,'__file__','?')}")
        print(f"   jaxlib @ {getattr(jaxlib,'__file__','?')}")
    except Exception as e:
        WARN.append(f"jaxlib import failed: {e!r}")

    # Detect PJRT packages
    has_plugin = u.find_spec("jax_cuda12_plugin") is not None
    has_pjrt   = u.find_spec("jax_cuda12_pjrt") is not None
    if has_plugin and has_pjrt:
        WARN.append("Both jax-cuda12-plugin and jax-cuda12-pjrt are installed (conflict).")
        _msg_box(
            "Conflict: Two JAX PJRT CUDA backends found",
            textwrap.dedent("""\
                You must keep exactly one of these:
                  • jax-cuda12-plugin  (for LOCAL /usr/local/cuda)
                  • jax-cuda12-pjrt    (bundled runtime)
                Recommended (in this image): keep the plugin and remove pjrt:
                  uv pip uninstall -y jax-cuda12-pjrt
            """),
        )

    # Detect pip NVIDIA CUDA stacks if using local plugin
    def _pip_freeze_contains(prefix: str) -> bool:
        try:
            out = subprocess.check_output([os.environ.get("UV_BIN","uv"), "pip", "freeze"], text=True)
        except Exception:
            try:
                out = subprocess.check_output([sys.executable, "-m", "pip", "freeze"], text=True)
            except Exception:
                return False
        return any(line.strip().startswith(prefix) for line in out.splitlines())

    if has_plugin:
        offending = [p for p in (
            "nvidia-cuda-runtime-cu12", "nvidia-cudnn-cu12", "nvidia-cublas-cu12",
            "nvidia-cusolver-cu12", "nvidia-cusparse-cu12", "nvidia-cufft-cu12",
            "nvidia-curand-cu12", "nvidia-nvtx-cu12", "nvidia-nvjitlink-cu12", "nvidia-cuda-cupti-cu12"
        ) if _pip_freeze_contains(p)]
        if offending:
            WARN.append(f"Local CUDA policy but pip NVIDIA libs present: {', '.join(offending)}")
            _msg_box(
                "Mixed CUDA stacks detected",
                "You're using the local PJRT plugin, but pip-installed NVIDIA CUDA libs are present.\n"
                "Remove them to avoid duplicate allocators:\n"
                "  uv pip uninstall -y " + " ".join(offending)
            )

    # Device probe
    try:
        devs = jax.devices()
        print(f"   jax {getattr(jax,'__version__','?')} devices: {devs}")
    except Exception as e:
        WARN.append(f"jax.devices() raised: {e!r}")
        _msg_box(
            "Action: Fix JAX GPU backend",
            textwrap.dedent("""\
                • Install the PJRT CUDA plugin (local CUDA policy):
                  uv pip install "jax[cuda12-local]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
                • Ensure only one backend (plugin XOR pjrt) is installed.
                • Avoid mixing pip 'nvidia-*' CUDA stacks with the system CUDA.
            """),
        )
        return

    # CPU-only hints
    gpu = [d for d in devs if "gpu" in str(d).lower() or "cuda" in str(d).lower()]
    if not gpu:
        WARN.append("JAX imported but reports CPU-only devices.")
        _msg_box(
            "Info: JAX is CPU-only right now",
            textwrap.dedent("""\
                Likely causes (check in order):
                1) CUDA plugin not installed in this venv:
                   uv pip install "jax[cuda12-local]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
                2) Backend conflict (both plugin and pjrt installed) — remove one.
                3) Mixed CUDA stacks (remove pip 'nvidia-*' libs when using local CUDA).
            """),
        )


def main():
    print("## Python & library diagnostics ##")
    print("Python:", sys.version.split()[0])
    print("sys.executable:", sys.executable)
    print("sys.prefix:", sys.prefix)
    print("VIRTUAL_ENV:", os.environ.get("VIRTUAL_ENV","<unset>"))
    print("PATH head:", os.environ.get("PATH","").split(":")[:3])

    if not sys.executable.startswith("/app/.venv/"):
        CRIT.append("Interpreter is not /app/.venv — uv env not active for this process")

    jlab_ok = _have("jupyterlab")
    print(f" - jupyterlab: {'OK' if jlab_ok else 'MISSING'}")

    torch_ok = _have("torch")
    print(f" - torch: {'OK' if torch_ok else 'MISSING'}")
    if torch_ok:
        try:
            import torch
            print("   torch", torch.__version__, "CUDA available:", torch.cuda.is_available())
        except Exception as e:
            WARN.append(f"torch import ok but CUDA probe errored: {e}")

    print(f" - jax: {'OK' if _have('jax') else 'MISSING'}")
    _probe_jax()

    if CRIT:
        _msg_box("Critical failures", "\n".join(f"• {m}" for m in CRIT))
        sys.exit(1)

    if WARN:
        _msg_box("Warnings (non-blocking)", "\n".join(f"• {m}" for m in WARN))

    print("✅ verify_env completed (warnings above are informational).")
    sys.exit(0)

if __name__ == "__main__":
    main()


Overwriting ../../.devcontainer/verify_env.py


In [8]:
%%writefile ../../scripts/test-gpu.sh
#!/usr/bin/env bash
set -euo pipefail
PROJECT=${1:-cancer_bayes_iris_env}
SERVICE=${2:-datascience}
CID=$(docker compose -p "$PROJECT" ps -q "$SERVICE")
[ -z "$CID" ] && { echo "no container"; exit 1; }

echo "== nvidia-smi ==" && docker exec "$CID" nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv,noheader,nounits

echo "== PyTorch ==" && docker exec "$CID" python - <<'PY'
import torch, gc
print("torch:", torch.__version__, "cuda:", torch.version.cuda)
print("cuda.is_available:", torch.cuda.is_available())
assert torch.cuda.is_available()
print("device:", torch.cuda.get_device_name(0))
x = torch.randn(1024,1024, device="cuda")
y = torch.randn(1024,1024, device="cuda")
_ = (x@y).sum().item()
del x,y; gc.collect(); torch.cuda.empty_cache()
print("PyTorch GPU: OK")
PY

echo "== JAX ==" && docker exec "$CID" python - <<'PY'
import jax, jax.numpy as jnp
print("jax:", jax.__version__)
print("devices:", jax.devices())
assert any("gpu" in str(d).lower() or "cuda" in str(d).lower() for d in jax.devices())
x=jnp.ones((1024,1024)); y=(x@x.T).sum(); _=y.block_until_ready()
print("JAX GPU: OK")
PY



Overwriting ../../scripts/test-gpu.sh


In [9]:
%%writefile ../../.devcontainer/scripts/test-gpu.sh
# PowerShell GPU Testing Script for Windows Docker Development
# ==========================================================
# This script provides Windows-native testing capabilities for GPU frameworks
# in Docker containers, optimized for NVIDIA Blackwell RTX 5080 setups.
#
# Usage:
#   .\scripts\test-gpu.ps1
#   .\scripts\test-gpu.ps1 -Verbose
#   .\scripts\test-gpu.ps1 -ContainerName "custom_container"

param(
  [string]$ComposeProject = "cancer_bayes_iris_env",
  [string]$Service = "datascience",
  [switch]$Verbose
)

function Out-Green($m){ Write-Host "✅ $m" -ForegroundColor Green }
function Out-Red($m){ Write-Host "❌ $m" -ForegroundColor Red }
function Out-Cyan($m){ Write-Host "ℹ️  $m" -ForegroundColor Cyan }

$cid = docker compose -p $ComposeProject ps -q $Service
if (-not $cid) { Out-Red "No container found"; exit 1 }
Out-Green "Container: $cid"

Out-Cyan "nvidia-smi…"
docker exec $cid nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv,noheader,nounits

Out-Cyan "PyTorch CUDA…"
$pt = @'
import torch, gc
print("torch:", torch.__version__, "cuda:", torch.version.cuda)
print("cuda.is_available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("device:", torch.cuda.get_device_name(0))
    x = torch.randn(1024,1024, device="cuda")
    y = torch.randn(1024,1024, device="cuda")
    _ = (x@y).sum().item()
    del x,y; gc.collect(); torch.cuda.empty_cache()
    print("PyTorch GPU: OK")
else:
    raise SystemExit(1)
'@
docker exec $cid python -c $pt; if ($LASTEXITCODE) { Out-Red "PyTorch failed"; exit 1 } else { Out-Green "PyTorch OK" }

Out-Cyan "JAX CUDA…"
$jx = @'
import jax, importlib.util as u
print("jax:", jax.__version__)
print("devices:", jax.devices())
g = [d for d in jax.devices() if "gpu" in str(d).lower() or "cuda" in str(d).lower()]
if not g: raise SystemExit(1)
import jax.numpy as jnp
x=jnp.ones((1024,1024)); y=(x@x.T).sum(); _=y.block_until_ready()
print("JAX GPU: OK")
'@
docker exec $cid python -c $jx; if ($LASTEXITCODE) { Out-Red "JAX failed"; exit 1 } else { Out-Green "JAX OK" }


Overwriting ../../.devcontainer/scripts/test-gpu.sh


In [10]:
%%writefile ../../.devcontainer/opening.sh
#!/usr/bin/env bash
set -Eeuo pipefail

log() { printf "\n[%s] %s\n" "$(date +'%F %T')" "$*" >&2; }
die() { printf "\n[ERROR] %s\n" "$*" >&2; exit 1; }

# Parse boolean-ish env values: 1/true/yes/on → 1, else 0
parse_env_bool() {
  case "${1:-}" in
    1|true|TRUE|yes|YES|on|ON) echo "1" ;;
    *) echo "0" ;;
  esac
}

# Load /workspace/.env if present, exporting its variables
read_dotenv_if_present() {
  if [ -f "/workspace/.env" ]; then
    log "Sourcing /workspace/.env into the environment"
    sed -i 's/\r$//' /workspace/.env || true
    set -a; . /workspace/.env; set +a
  fi
}

# Normalize line endings for shell scripts
normalize_line_endings() {
  log "Normalizing CRLF -> LF for .devcontainer/*.sh"
  find .devcontainer -maxdepth 1 -type f -name "*.sh" -print0 | \
    xargs -0 -I{} bash -lc 'sed -i "s/\r$//" "{}"'
}

# Ensure git safe.directory
fix_git_safety() {
  if git rev-parse --show-toplevel >/dev/null 2>&1; then
    local root="$(git rev-parse --show-toplevel || echo /workspace)"
    log "Marking ${root} as a safe.directory for git."
    git config --global --add safe.directory "${root}" || true
  fi
}

# Ensure uv targets the project venv
ensure_uv_env() {
  export UV_PROJECT_ENVIRONMENT="${UV_PROJECT_ENVIRONMENT:-/app/.venv}"
  log "UV_PROJECT_ENVIRONMENT=${UV_PROJECT_ENVIRONMENT}"
  if [[ ! -d "${UV_PROJECT_ENVIRONMENT}" ]]; then
    log "Creating uv venv at ${UV_PROJECT_ENVIRONMENT}"
    uv venv "${UV_PROJECT_ENVIRONMENT}" --python "3.10" --prompt "cancer_bayes_iris_env"
  fi
  echo ". ${UV_PROJECT_ENVIRONMENT}/bin/activate" > /etc/profile.d/10-uv-activate.sh
}

# Sync core project dependencies (prefer root pyproject.toml)
sync_project_deps() {
  # Prefer root pyproject for universal local+container dev
  if [[ -f "/workspace/pyproject.toml" ]]; then
    log "Syncing project deps from /workspace (frozen)"
    (cd /workspace && uv sync --frozen --no-dev) || {
      log "Lock out-of-date; refreshing from /workspace…"
      (cd /workspace && uv sync --no-dev && uv lock)
    }
  elif [[ -f "/workspace/.devcontainer/pyproject.toml" ]]; then
    log "Syncing project deps from .devcontainer (legacy fallback)"
    (cd /workspace/.devcontainer && uv sync --frozen --no-dev) || {
      log "Lock out-of-date; refreshing from .devcontainer…"
      (cd /workspace/.devcontainer && uv sync --no-dev && uv lock)
    }
  else
    die "No pyproject.toml found at /workspace or .devcontainer"
  fi
}

# Simplified memory management for RTX 5080
setup_memory_management() {
  log "Setting up RTX 5080 memory management..."

  # Verify jemalloc
  if python -c "import ctypes; ctypes.CDLL('/usr/lib/x86_64-linux-gnu/libjemalloc.so.2')" 2>/dev/null; then
    log "jemalloc memory allocator loaded successfully"
  else
    log "jemalloc not available - using system malloc"
    unset LD_PRELOAD
  fi

  # Essential memory settings only (removed complex variables)
  export MALLOC_ARENA_MAX=1
  export MALLOC_TCACHE_MAX=0
  export PYTORCH_NO_CUDA_MEMORY_CACHING=1
  export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512,expandable_segments:True
  export XLA_PYTHON_CLIENT_PREALLOCATE=false
  export XLA_PYTHON_CLIENT_MEM_FRACTION=0.25

  log "Memory management configured"
}

# GPU Framework Installation (JAX + PyTorch only, TensorFlow removed)
install_gpu_frameworks() {
  log "Installing GPU frameworks (PyTorch + JAX only)..."

  read_dotenv_if_present

  # Clear JAX platform forcing
  unset JAX_PLATFORM_NAME || true
  unset JAX_PLATFORMS || true

  # Remove conflicting NVIDIA packages to prevent double loads
  log "Cleaning up conflicting NVIDIA packages..."
  uv pip uninstall -y \
    nvidia-cublas-cu12 nvidia-cuda-runtime-cu12 nvidia-cudnn-cu12 \
    nvidia-cuda-cupti-cu12 nvidia-cusolver-cu12 nvidia-cusparse-cu12 \
    nvidia-nvjitlink-cu12 nvidia-nvtx-cu12 nvidia-cufft-cu12 nvidia-curand-cu12 \
    || true

  # PyTorch with CUDA 12.8 support
  if ! python -c "import torch; assert torch.cuda.is_available()" 2>/dev/null; then
    log "Installing PyTorch nightly cu128..."
    uv pip install --no-cache-dir --pre torch torchvision torchaudio \
      --index-url "https://download.pytorch.org/whl/nightly/cu128"
  else
    log "PyTorch CUDA already available"
  fi

  # JAX with CUDA support  
  log "Installing JAX with CUDA support..."
  uv pip uninstall -y jax jaxlib jax-cuda12-plugin jax-cuda12-pjrt || true
  uv pip install --no-cache-dir "jax[cuda12-local]" \
    -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
}

# Framework initialization (PyTorch + JAX only)
initialize_frameworks() {
  log "Initializing PyTorch and JAX..."

  python3 -c "
import os, gc
print('Initializing JAX...')
try:
    import jax, jax.numpy as jnp
    devs = jax.devices()
    print('   JAX devices:', devs)
    x = jnp.ones((128,128)); y = (x @ x.T).sum(); _ = y.block_until_ready()
    print('   JAX small matmul: OK')
except Exception as e:
    print('   JAX init failed:', e)

print('Initializing PyTorch...')
try:
    import torch
    print('   torch:', torch.__version__, 'CUDA avail:', torch.cuda.is_available())
    if torch.cuda.is_available():
        print('   device:', torch.cuda.get_device_name(0))
        a = torch.randn(128,128, device='cuda')
        b = torch.randn(128,128, device='cuda')
        _ = (a@b).sum().item()
        print('   PyTorch small matmul: OK')
except Exception as e:
    print('   PyTorch init failed:', e)

gc.collect()
"
}

# Register Jupyter kernel
register_kernel() {
  log "Registering Jupyter ipykernel..."
  python -c "
import json, sys, subprocess
name = 'cancer_bayes_iris_env'
display = 'Python (cancer_bayes_iris_env)'
try:
    subprocess.check_call([sys.executable, '-m', 'ipykernel', 'install',
                           '--sys-prefix', '--name', name, '--display-name', display])
    print('Kernel registered:', display)
except subprocess.CalledProcessError as e:
    print('Kernel registration failed:', e)
    sys.exit(1)
"
}

# Simplified verification (PyTorch + JAX only)
verify_frameworks() {
  log "Verifying frameworks..."

  local failed=0

  # PyTorch verification
  if python -c "
import torch, gc
assert torch.cuda.is_available(), 'CUDA not available'
for _ in range(3):
    x = torch.randn(1000,1000, device='cuda')
    del x; torch.cuda.empty_cache(); gc.collect()
print('PyTorch CUDA verification passed')
" 2>/dev/null; then
    log "PyTorch verification: PASSED"
  else
    log "PyTorch verification: FAILED"
    failed=$((failed+1))
  fi

  # JAX verification  
  if python -c "
import jax, jax.numpy as jnp, gc
devices = jax.devices()
assert any('gpu' in str(d).lower() or 'cuda' in str(d).lower() for d in devices), 'No GPU devices'
for _ in range(3):
    x = jnp.ones((1000,1000)); del x; gc.collect()
print('JAX CUDA verification passed')
" 2>/dev/null; then
    log "JAX verification: PASSED"
  else
    log "JAX verification: FAILED"  
    failed=$((failed+1))
  fi

  if [ "${failed}" -gt 0 ]; then
    log "Some frameworks failed verification but continuing..."
  else
    log "All frameworks verified successfully"
  fi
}

# MAIN EXECUTION
main() {
  normalize_line_endings
  read_dotenv_if_present
  fix_git_safety
  ensure_uv_env
  setup_memory_management

  if [[ "${1:-}" == "--verify-only" ]]; then
    verify_frameworks || true
    return 0
  fi

  sync_project_deps
  install_gpu_frameworks
  initialize_frameworks  
  register_kernel
  verify_frameworks
  log "Setup completed successfully"
}

main "$@"


Overwriting ../../.devcontainer/opening.sh


In [11]:
%%writefile ../../.devcontainer/rebuild.sh
#!/bin/bash
set -euo pipefail

# DevContainer GPU Setup Rebuild Script
# This script helps rebuild the DevContainer with all the latest fixes

echo "🔧 DevContainer GPU Setup Rebuild Script"
echo "========================================"
echo ""

# Function for logging
log() {
    echo "[$(date '+%Y-%m-%d %H:%M:%S')] $1"
}

# Function for status indicators
status_ok() { echo "✅ $1"; }
status_warn() { echo "⚠️  $1"; }
status_error() { echo "❌ $1"; }

# Check if we're in the right directory
if [ ! -f ".devcontainer/devcontainer.json" ]; then
    status_error "This script must be run from the project root directory"
    echo "Please run: cd /path/to/your/project && bash .devcontainer/rebuild.sh"
    exit 1
fi

log "Starting DevContainer rebuild process..."

# Step 1: Stop existing containers
log "Step 1: Stopping existing containers..."
if docker compose -p cancer_bayes_iris_env ps -q >/dev/null 2>&1; then
    docker compose -p cancer_bayes_iris_env down
    status_ok "Existing containers stopped"
else
    status_warn "No existing containers found"
fi

# Step 2: Clean up Docker resources
log "Step 2: Cleaning up Docker resources..."
docker system prune -f --volumes
status_ok "Docker resources cleaned"

# Step 3: Rebuild with no cache
log "Step 3: Rebuilding DevContainer with no cache..."
docker compose -p cancer_bayes_iris_env build --no-cache datascience
status_ok "DevContainer rebuilt successfully"

# Step 4: Start the container
log "Step 4: Starting DevContainer..."
docker compose -p cancer_bayes_iris_env up -d
status_ok "DevContainer started"

# Step 5: Wait for container to be ready
log "Step 5: Waiting for container to be ready..."
sleep 15

# Step 6: Check container status
log "Step 6: Checking container status..."
if docker compose -p cancer_bayes_iris_env ps | grep -q "Up"; then
    status_ok "Container is running"
else
    status_error "Container failed to start"
    echo "Check logs with: docker compose -p cancer_bayes_iris_env logs -f"
    exit 1
fi

# Step 7: Run setup script inside container
log "Step 7: Running setup script inside container..."
CONTAINER_NAME=$(docker compose -p cancer_bayes_iris_env ps -q datascience)
if [ -n "$CONTAINER_NAME" ]; then
    docker exec -it "$CONTAINER_NAME" bash -c "cd /workspace && bash .devcontainer/setup.sh"
    status_ok "Setup script completed"
else
    status_error "Could not find container"
    exit 1
fi

# Step 8: Verify GPU access
log "Step 8: Verifying GPU access..."
docker exec -it "$CONTAINER_NAME" bash -c "nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv,noheader,nounits" || {
    status_warn "GPU not accessible (this may be expected if no GPU is available)"
}

# Step 9: Test Python imports
log "Step 9: Testing Python imports..."
docker exec -it "$CONTAINER_NAME" bash -c "python - <<'PY'
import importlib, os
mods = ['pymc','torch','jax']
ok = True
for m in mods:
    try: importlib.import_module(m); print('✅', m, 'import OK')
    except Exception as e: print('❌', m, 'failed:', e); ok = False
if not ok: raise SystemExit(1)
print('✅ All key packages imported successfully')
PY" || { status_error "Some packages failed to import"; }

# Step 10: Run basic GPU verification
log "Step 10: Running basic GPU verification..."
docker exec -it "$CONTAINER_NAME" bash -c "nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv,noheader,nounits" || {
    status_warn "GPU verification failed - check the output above"
}

# Step 11: Test framework imports (TensorFlow optional)
log "Step 11: Testing framework imports..."
docker exec -it "$CONTAINER_NAME" bash -c "python - <<'PY'
import importlib, os, sys
install_tf = os.getenv('INSTALL_TF','0').lower() in ('1','true','yes','on')
to_check = ['torch','jax'] + (['tensorflow'] if install_tf else [])
ok = True
for m in to_check:
    try: importlib.import_module(m); print('✅', m, 'import OK')
    except Exception as e: print('❌', m, 'failed:', e); ok = False
if not ok: raise SystemExit(1)
print('✅ Requested frameworks imported successfully')
PY" || { status_warn "A requested framework failed to import - check logs"; }

# Step 12: Check memory management
log "Step 12: Checking memory management setup..."
docker exec -it "$CONTAINER_NAME" bash -c "
echo '🔍 Checking jemalloc installation...'
if [ -f '/usr/lib/x86_64-linux-gnu/libjemalloc.so.2' ]; then
    echo '✅ jemalloc library found'
else
    echo '❌ jemalloc library not found'
fi

echo '🔍 Checking memory environment variables...'
env | grep -E '(LD_PRELOAD|MALLOC|PYTORCH|XLA|TF_FORCE)' || echo 'No memory management variables found'

echo '🔍 Checking memory allocator in use...'
python -c \"
import ctypes
try:
    ctypes.CDLL('/usr/lib/x86_64-linux-gnu/libjemalloc.so.2')
    print('✅ jemalloc is loaded')
except:
    print('❌ jemalloc not loaded')
\"
" || {
    status_warn "Memory management check failed"
}

echo ""
echo "🎉 DevContainer rebuild completed!"
echo ""
echo "Next steps:"
echo "1. Open VS Code and press F1 → 'Dev Containers: Reopen in Container'"
echo "2. Or connect to Jupyter Lab at: http://localhost:8890 (token: jupyter)"
echo "3. Run diagnostics: bash .devcontainer/troubleshoot.sh"
echo "4. Test GPU from host: ./scripts/test-gpu.ps1 (Windows) or ./scripts/test-gpu.sh (Linux/macOS)"
echo ""
echo "Useful commands:"
echo "  View logs: docker compose -p cancer_bayes_iris_env logs -f"
echo "  Enter container: docker exec -it $CONTAINER_NAME bash"
echo "  Stop container: docker compose -p cancer_bayes_iris_env down"
echo "  Test GPU: ./scripts/test-gpu.ps1 (Windows) or ./scripts/test-gpu.sh (Linux/macOS)"



Overwriting ../../.devcontainer/rebuild.sh


In [12]:
%%writefile ../../.devcontainer/setup.sh
#!/bin/bash
set -euo pipefail

# Error handling
handle_error() {
    echo "❌ Error at line: ${1}"
    exit 1
}
trap 'handle_error ${LINENO}' ERR

log() {
    echo "[$(date '+%Y-%m-%d %H:%M:%S')] $1"
}

log "🚀 Starting DevContainer setup..."

# Verify environment
log "📋 Environment check..."
echo "Python: $(python --version)"
echo "uv: $(uv --version)"
echo "Working directory: $(pwd)"
echo "VIRTUAL_ENV: ${VIRTUAL_ENV:-<unset>}"

# Ensure we're in workspace
cd /workspace

# Check if dependencies need to be installed
if ! python -c "import pymc" 2>/dev/null; then
    log "📦 Installing project dependencies..."
    
    # Prefer root pyproject.toml for universal local+container dev
    if [ -f "pyproject.toml" ]; then
        log "Using workspace pyproject.toml"
        uv sync --frozen || uv sync
    elif [ -f ".devcontainer/pyproject.toml" ]; then
        log "Using .devcontainer/pyproject.toml (fallback)"
        cd .devcontainer
        uv sync --frozen || uv sync
        cd ..
    else
        log "❌ No pyproject.toml found in /workspace or .devcontainer"
        exit 1
    fi
else
    log "✅ Dependencies already installed"
fi

# Ensure ipykernel is installed and registered
log "🔧 Setting up Jupyter kernel..."
python -m ipykernel install --name uv-app-venv --display-name "Python (uv /app/.venv)" --user || true

# Install PyJAGS (optional)
log "📊 Installing PyJAGS (optional)..."
CPPFLAGS='-include cstdint' uv pip install --no-build-isolation pyjags==1.3.8 || {
    log "⚠️  PyJAGS installation failed (optional)"
}

# Verify GPU access
log "🔍 Checking GPU..."
if command -v nvidia-smi >/dev/null 2>&1; then
    nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv,noheader,nounits || true
fi

# Run verification
log "🔬 Running environment verification..."
python .devcontainer/verify_env.py || true

log "✅ Setup completed"






Overwriting ../../.devcontainer/setup.sh


In [13]:
%%writefile ../../.devcontainer/postcreate.sh
#!/usr/bin/env bash
set -euo pipefail
echo "## postCreate (idempotent) ##"

PROJECT_DIR=${PROJECT_DIR:-/workspace}
DEV_DIR="${PROJECT_DIR}/.devcontainer"
TARGET_DIR="${PROJECT_DIR}"
if [ ! -f "${TARGET_DIR}/pyproject.toml" ]; then
  TARGET_DIR="${DEV_DIR}"
fi

# Ensure .env exists first
bash "${DEV_DIR}/setup_env.sh"

export UV_PROJECT_ENVIRONMENT=/app/.venv

# Dependency hash sentinel
mkdir -p "${DEV_DIR}"
dep_hash="$(
  { [ -f "${TARGET_DIR}/pyproject.toml" ] && cat "${TARGET_DIR}/pyproject.toml"; } 2>/dev/null
  { [ -f "${TARGET_DIR}/uv.lock" ] && cat "${TARGET_DIR}/uv.lock"; } 2>/dev/null
  ) | sha256sum | awk '{print $1}'
"
sentinel="${DEV_DIR}/.postcreate.hash"
prev_hash="$( [ -f "${sentinel}" ] && cat "${sentinel}" || echo "" )"

uv --version || true

if [ "${dep_hash}" != "${prev_hash}" ]; then
  echo "[postCreate] Dependency hash changed or first run → uv sync"
  (uv sync --frozen || uv sync)
  echo "${dep_hash}" > "${sentinel}"
else
  echo "[postCreate] Dependencies up-to-date; skipping uv sync"
fi

python - <<'PY'
import sys, subprocess
print('[postCreate] Ensuring ipykernel present for', sys.executable)
try:
    import ipykernel  # noqa
except Exception:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'ipykernel'])
subprocess.check_call([
    sys.executable, '-m', 'ipykernel', 'install',
    '--name', 'uv-app-venv',
    '--display-name', 'Python (uv /app/.venv)',
    '--user'
])
print('[postCreate] ipykernel installed/updated as uv-app-venv')
PY

echo "[postCreate] Optional PyJAGS"
CPPFLAGS='-include cstdint' uv pip install --no-build-isolation pyjags==1.3.8 || true

python "${DEV_DIR}/verify_env.py" || true
echo "## postCreate DONE ##"


Overwriting ../../.devcontainer/postcreate.sh


In [14]:
%%writefile ../../.devcontainer/postCreate_rtx5080.sh
#!/usr/bin/env bash
set -euo pipefail

echo "## postCreate RTX 5080 Optimized (idempotent) ##"

PROJECT_DIR=${PROJECT_DIR:-/workspace}
DEV_DIR="${PROJECT_DIR}/.devcontainer"
TARGET_DIR="${PROJECT_DIR}"
if [ ! -f "${TARGET_DIR}/pyproject.toml" ]; then
  TARGET_DIR="${DEV_DIR}"
fi

# Ensure .env exists first
bash "${DEV_DIR}/setup_env.sh"

export UV_PROJECT_ENVIRONMENT=/app/.venv

# --- CRITICAL: Environment hygiene for JAX ---
echo "[postCreate] Clearing JAX platform forcing for RTX 5080 compatibility..."
unset JAX_PLATFORM_NAME || true
unset JAX_PLATFORMS || true

# --- Memory management for RTX 5080 ---
export TF_FORCE_GPU_ALLOW_GROWTH=true
export TF_GPU_ALLOCATOR=cuda_malloc_async
export XLA_PYTHON_CLIENT_PREALLOCATE=false
export XLA_PYTHON_CLIENT_ALLOCATOR=platform
export XLA_PYTHON_CLIENT_MEM_FRACTION=0.25

echo "[postCreate] Using uv=$(uv --version || true)"

# Upgrade pip tooling inside uv venv for reliability
uv pip install --upgrade pip wheel setuptools

# Dependency hash sentinel
mkdir -p "${DEV_DIR}"
dep_hash="$(
  { [ -f "${TARGET_DIR}/pyproject.toml" ] && cat "${TARGET_DIR}/pyproject.toml"; } 2>/dev/null
  { [ -f "${TARGET_DIR}/uv.lock" ] && cat "${TARGET_DIR}/uv.lock"; } 2>/dev/null
  ) | sha256sum | awk '{print $1}'
"
sentinel="${DEV_DIR}/.postcreate.hash"
prev_hash="$( [ -f "${sentinel}" ] && cat "${sentinel}" || echo "" )"

if [ "${dep_hash}" != "${prev_hash}" ]; then
  echo "[postCreate] Dependency hash changed or first run → uv sync"
  (uv sync --frozen || uv sync)
  echo "${dep_hash}" > "${sentinel}"
else
  echo "[postCreate] Dependencies up-to-date; skipping uv sync"
fi

# ---- PyTorch (CUDA 12.8) ----
echo "[postCreate] Installing PyTorch nightly for RTX 5080 (CUDA 12.8)..."
uv pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128

# ---- JAX (CUDA 12) ----
echo "[postCreate] Installing JAX with CUDA 12 support for RTX 5080..."
# Remove any existing JAX installations to avoid conflicts
uv pip uninstall -y jax jaxlib jax-cuda12-plugin jax-cuda12-pjrt || true
# Install JAX with proper CUDA 12 support
uv pip install -U "jax[cuda12]"

# ---- TensorFlow removed for efficiency ----
echo "[postCreate] TensorFlow installation skipped (removed for efficiency)"

# ---- Quick version check ----
python - <<'PY'
import os, importlib.util
print("=== RTX 5080 Framework Versions ===")
try:
    import torch; print("torch:", torch.__version__, "cuda:", torch.version.cuda)
except Exception as e: print("torch import failed:", e)
try:
    import jax, jaxlib; print("jax:", jax.__version__, "jaxlib:", jaxlib.__version__)
except Exception as e: print("jax import failed:", e)
# TensorFlow removed for efficiency
print("tf: removed for efficiency")
print("JAX_PLATFORM_NAME =", os.getenv("JAX_PLATFORM_NAME"))
print("===================================")
PY

# ---- Ipykernel setup ----
python - <<'PY'
import sys, subprocess
print('[postCreate] Ensuring ipykernel present for', sys.executable)
try:
    import ipykernel  # noqa
except Exception:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'ipykernel'])
subprocess.check_call([
    sys.executable, '-m', 'ipykernel', 'install',
    '--name', 'uv-app-venv',
    '--display-name', 'Python (uv /app/.venv)',
    '--user'
])
print('[postCreate] ipykernel installed/updated as uv-app-venv')
PY

echo "[postCreate] Optional PyJAGS"
CPPFLAGS='-include cstdint' uv pip install --no-build-isolation pyjags==1.3.8 || true

# ---- Framework verification (non-fatal) ----
echo "[postCreate] Running framework verification..."
python "${DEV_DIR}/verify_env.py" || {
    echo "[postCreate] Framework verification had issues - check logs above"
    echo "[postCreate] This is non-fatal - container will continue"
}

echo "## postCreate RTX 5080 DONE ##"




Overwriting ../../.devcontainer/postCreate_rtx5080.sh


In [15]:
%%writefile ../../.devcontainer/rtx5080_repair.sh
#!/usr/bin/env bash
set -euo pipefail

echo "🔧 RTX 5080 GPU Framework Repair Script"
echo "======================================="
echo "This script will fix JAX and TensorFlow issues for RTX 5080"
echo ""

# Check if we're in the right environment
if [[ ! -f "/app/.venv/bin/activate" ]]; then
    echo "❌ Error: This script must be run inside the container with uv environment"
    exit 1
fi

# Activate the environment
source /app/.venv/bin/activate

echo "📋 Current environment:"
echo "   UV_PROJECT_ENVIRONMENT: ${UV_PROJECT_ENVIRONMENT:-<unset>}"
echo "   Python: $(python --version)"
echo "   uv: $(uv --version)"
echo ""

# --- Step 1: Environment Cleanup ---
echo "🧹 Step 1: Cleaning up environment variables..."
unset JAX_PLATFORM_NAME || true
unset JAX_PLATFORMS || true

# Set optimal environment variables
export TF_FORCE_GPU_ALLOW_GROWTH=true
export TF_GPU_ALLOCATOR=cuda_malloc_async
export XLA_PYTHON_CLIENT_PREALLOCATE=false
export XLA_PYTHON_CLIENT_ALLOCATOR=platform
export XLA_PYTHON_CLIENT_MEM_FRACTION=0.25

echo "   ✅ Environment variables configured"
echo ""

# --- Step 2: JAX Repair ---
echo "🧠 Step 2: Repairing JAX..."
echo "   Removing existing JAX installations..."

# Remove all JAX-related packages
uv pip uninstall -y jax jaxlib jax-cuda12-plugin jax-cuda12-pjrt || true

echo "   Installing JAX with CUDA 12 support..."
uv pip install -U "jax[cuda12]"

echo "   ✅ JAX installation completed"
echo ""

# --- Step 3: TensorFlow removed for efficiency ---
echo "🔢 Step 3: TensorFlow removed for efficiency..."
echo "   TensorFlow installation skipped to improve build speed and reduce conflicts"
echo ""

# --- Step 4: Verification ---
echo "🔍 Step 4: Verifying installations..."

python - <<'PY'
import os, importlib.util
print("=== Installation Verification ===")

# Check JAX
try:
    import jax, jaxlib
    print(f"✅ JAX: {jax.__version__}")
    print(f"✅ JAXlib: {jaxlib.__version__}")

    # Check for CUDA plugin
    has_cuda = importlib.util.find_spec("jax_cuda12_plugin") or importlib.util.find_spec("jax_cuda12_pjrt")
    print(f"✅ JAX CUDA plugin: {'Yes' if has_cuda else 'No'}")

    # Check devices
    try:
        devices = jax.devices()
        gpu_devices = [d for d in devices if 'gpu' in str(d).lower() or 'cuda' in str(d).lower()]
        print(f"✅ JAX GPU devices: {len(gpu_devices)} found")
        for d in gpu_devices:
            print(f"   - {d}")
    except Exception as e:
        print(f"❌ JAX device check failed: {e}")

except Exception as e:
    print(f"❌ JAX verification failed: {e}")

# TensorFlow removed for efficiency
print("✅ TensorFlow: removed for efficiency")

# Check PyTorch
try:
    import torch
    print(f"✅ PyTorch: {torch.__version__}")
    print(f"✅ PyTorch CUDA: {torch.version.cuda}")
    print(f"✅ PyTorch CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"✅ PyTorch device: {torch.cuda.get_device_name(0)}")
except Exception as e:
    print(f"❌ PyTorch verification failed: {e}")

print("=== Environment Variables ===")
print(f"JAX_PLATFORM_NAME: {os.environ.get('JAX_PLATFORM_NAME', '<unset>')}")
print(f"TF_FORCE_GPU_ALLOW_GROWTH: {os.environ.get('TF_FORCE_GPU_ALLOW_GROWTH', '<unset>')}")
print("=============================")
PY

echo ""
echo "🎉 Repair completed!"
echo ""
echo "💡 Next steps:"
echo "   1. Test the frameworks with: python .devcontainer/enhanced_gpu_test_functions.py"
echo "   2. If issues persist, check the diagnostic output above"
echo "   3. For TensorFlow PTX errors, consider building from source with CUDA 12.8+"
echo ""
echo "🔗 Useful commands:"
echo "   # Test all frameworks"
echo "   python .devcontainer/enhanced_gpu_test_functions.py"
echo ""
echo "   # Quick JAX test"
echo "   python -c \"import jax; print('JAX devices:', jax.devices())\""
echo ""
echo "   # Quick TensorFlow test"
echo "   python -c \"import tensorflow as tf; print('TF GPUs:', tf.config.list_physical_devices('GPU'))\""
echo ""



Overwriting ../../.devcontainer/rtx5080_repair.sh


In [16]:
%%writefile ../../.devcontainer/enhanced_gpu_test_functions.py
#!/usr/bin/env python3
"""
Enhanced GPU Test Functions for RTX 5080 (PyTorch + JAX Only)
============================================================

Drop-in replacements for the test functions in your notebook.
These provide comprehensive diagnostics and actionable guidance for RTX 5080 issues.

Usage:
    # Import and use in your notebook
    from .devcontainer.enhanced_gpu_test_functions import check_jax, check_pytorch, comprehensive_gpu_test

    # Or run standalone
    python .devcontainer/enhanced_gpu_test_functions.py
"""

import os
import sys
import gc
import textwrap
import subprocess
import importlib.util


def _run(cmd):
    """
    Run a command and return (success, output).

    Args:
        cmd: Command to run

    Returns:
        tuple: (success: bool, output: str)
    """
    try:
        out = subprocess.check_output(cmd, stderr=subprocess.STDOUT, text=True)
        return True, out.strip()
    except Exception as e:
        return False, f"{type(e).__name__}: {e}"


def check_jax():
    """
    Enhanced JAX GPU test with backend detection and Blackwell compatibility.

    This function provides:
    - Detection of CUDA backend plugins vs PJRT runtime
    - Identification of mixed backend conflicts
    - GPU device enumeration and computation testing
    - Clear error messages for common configuration issues
    - Actionable hints for RTX 5080 compatibility
    """
    import os, importlib.util, textwrap
    print("=== JAX ===")
    try:
        import jax
        print(f"jax.__version__={jax.__version__}")
        try:
            import jaxlib
            print(f"jaxlib.__version__={jaxlib.__version__}")
        except Exception as e:
            print("Could not import jaxlib:", repr(e))

        forced = os.environ.get("JAX_PLATFORM_NAME"), os.environ.get("JAX_PLATFORMS")
        print(f"JAX_PLATFORM_NAME={forced[0]!r} JAX_PLATFORMS={forced[1]!r}")

        # Is a CUDA PJRT plugin present?
        has_cuda_extra = importlib.util.find_spec("jax_cuda12_plugin") or importlib.util.find_spec("jax_cuda12_pjrt")
        print(f"JAX CUDA plugin present? {bool(has_cuda_extra)}")

        try:
            devs = jax.devices()
            if not devs:
                print("No JAX devices found.")
            else:
                for d in devs:
                    print(f"Device: kind={getattr(d,'device_kind',None)}, platform={d.platform}, id={d.id}")
            # Smoke test
            from jax import numpy as jnp
            x = jnp.ones((1024,1024), dtype=jnp.float32)
            y = (x @ x.T).sum()
            _ = y.block_until_ready()
            print("small matmul check (JAX default backend): OK")
        except RuntimeError as re:
            msg = str(re)
            print("jax.devices() raised RuntimeError:", msg)
            if "Unknown backend: 'gpu'" in msg:
                print(textwrap.dedent("""
                    HINT: GPU was forced but CUDA-enabled jaxlib is missing.
                    Fix:
                      1) unset JAX_PLATFORM_NAME / JAX_PLATFORMS
                      2) install GPU wheels (CUDA12):  uv pip install -U "jax[cuda12]"
                """).strip())
        except Exception as e:
            print("jax.devices() failed:", repr(e))
    except Exception as e:
        print("JAX import failed:", repr(e))
    print("===========\n")


def check_pytorch():
    """
    Enhanced PyTorch GPU test with memory management.

    This function provides:
    - CUDA availability and device information
    - Memory cleanup after computation
    - Version and capability reporting
    - RTX 5080 specific compatibility checks
    """
    print("=== PyTorch ===")
    try:
        import torch
        print(f"torch.__version__={torch.__version__}")
        print(f"torch.version.cuda={torch.version.cuda}")
        print(f"torch.cuda.is_available()={torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"torch.cuda.device_count()={torch.cuda.device_count()}")
            print(f"current device name: {torch.cuda.get_device_name(0)}")

            # Test computation with memory cleanup
            x = torch.randn(1024, 1024, device="cuda")
            y = torch.randn(1024, 1024, device="cuda")
            z = (x @ y).sum().item()
            del x, y
            gc.collect()
            torch.cuda.empty_cache()
            print("small matmul check (PyTorch CUDA): OK")
        else:
            print("CUDA not available in PyTorch.")
    except Exception as e:
        print("PyTorch check failed:", repr(e))
    print("==============\n")


def comprehensive_gpu_test():
    """
    Run comprehensive GPU framework testing (PyTorch + JAX only).

    This function tests PyTorch and JAX frameworks
    and provides a summary of results with specific guidance for failures.

    Returns:
        bool: True if all frameworks passed, False otherwise
    """
    print("🧪 COMPREHENSIVE GPU FRAMEWORK TEST (PyTorch + JAX Only)")
    print("=" * 60)
    print("Testing GPU support for PyTorch and JAX...")
    print("Optimized for NVIDIA Blackwell RTX 5080 / CUDA 12.8")
    print("TensorFlow removed for faster builds and cleaner environment")
    print("=" * 60)

    # Test frameworks
    check_pytorch()
    check_jax()  

    print("=" * 60)
    print("✅ Comprehensive GPU test completed!")
    print("Check output above for any framework-specific issues.")
    print("=" * 60)

    return True


def main():
    """Main entry point for standalone execution."""
    import os

    # Show environment snapshot
    print("🔧 GPU ENVIRONMENT SNAPSHOT")
    print("=" * 30)
    env_vars = [
        "JAX_PLATFORM_NAME", "JAX_PLATFORMS", "CUDA_VISIBLE_DEVICES",
        "XLA_FLAGS", "NVIDIA_VISIBLE_DEVICES", "NVIDIA_DRIVER_CAPABILITIES",
        "PYTORCH_CUDA_ALLOC_CONF"
    ]
    for var in env_vars:
        value = os.environ.get(var, "<unset>")
        print(f"{var}={value}")
    print("=" * 30)
    print()

    # Run comprehensive test
    comprehensive_gpu_test()


if __name__ == "__main__":
    main()



Overwriting ../../.devcontainer/enhanced_gpu_test_functions.py


In [17]:
%%writefile ../../.devcontainer/gpu_bootstrap.sh
#!/usr/bin/env bash
set -euxo pipefail

echo "[gpu-bootstrap] BEGIN"

PY="/app/.venv/bin/python"
export UV_PROJECT_ENVIRONMENT="/app/.venv"

handle_error() {
  echo "❌ GPU bootstrap error at line: ${1}"
  exit 1
}
trap 'handle_error ${LINENO}' ERR

log() { echo "[$(date '+%Y-%m-%d %H:%M:%S')] [gpu-bootstrap] $1"; }
_have_cmd() { command -v "$1" >/dev/null 2>&1; }

seed_pip() {
  if ! "$PY" -m pip --version >/dev/null 2>&1; then
    log "Seeding pip (ensurepip)"
    "$PY" -m ensurepip --upgrade || true
  fi
}

PIP() {
  if _have_cmd uv; then uv pip "$@"; else seed_pip; "$PY" -m pip "$@"; fi
}
PIP_SHOW() {
  if _have_cmd uv; then uv pip show "$@" || true; else seed_pip; "$PY" -m pip show "$@" || true; fi
}

pick_torch_index() {
  local tag="${CUDA_TAG:-12.8.0}"
  case "$tag" in
    12.1* ) echo "cu121" ;;
    12.4* ) echo "cu124" ;;
    12.5*|12.6*|12.7*|12.8*|12.9* ) echo "cu128" ;;
    * ) echo "cu128" ;;
  esac
}

log "Env:"
echo "  whoami=$(whoami)"
echo "  PY=$PY"
echo "  UV_PROJECT_ENVIRONMENT=${UV_PROJECT_ENVIRONMENT}"
echo "  CUDA_TAG=${CUDA_TAG:-12.8.0}"

$PY - <<'PY'
import sys, os
print("[gpu-bootstrap] sys.executable:", sys.executable)
print("[gpu-bootstrap] VIRTUAL_ENV:", os.environ.get("VIRTUAL_ENV","<unset>"))
for m in ("torch","jax","jaxlib"):
    try:
        mod = __import__(m)
        print(f"[gpu-bootstrap] pre: import {m}: OK from", getattr(mod,"__file__","?"))
    except Exception as e:
        print(f"[gpu-bootstrap] pre: import {m}: FAIL -> {e.__class__.__name__}: {e}")
PY

# GPU availability
if _have_cmd nvidia-smi; then
  log "nvidia-smi present"
  nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv,noheader,nounits || true
else
  log "No nvidia-smi; GPU likely not available"
fi

unset JAX_PLATFORM_NAME || true

# --- 1) Ensure PyTorch (GPU) -------------------------------------------------
log "Checking PyTorch CUDA availability..."
if $PY -c "import torch, sys; sys.exit(0 if torch.cuda.is_available() else 1)" 2>/dev/null; then
  log "✅ PyTorch CUDA available"
else
  if _have_cmd nvidia-smi; then
    IDX="$(pick_torch_index)"
    log "Installing PyTorch (${IDX})"
    PIP install --no-cache-dir torch torchvision torchaudio --index-url "https://download.pytorch.org/whl/${IDX}"
  else
    log "⚠️  Skipping PyTorch GPU install (no nvidia-smi)"
  fi
fi

# --- 2) Enforce single JAX PJRT backend (prefer local plugin) ---------------
log "Reconciling JAX backend packages..."
HAS_PLUGIN=0
HAS_PJRT=0
$PY - <<'PY'
import importlib.util, sys
def have(pkg):
    return importlib.util.find_spec(pkg) is not None
print("has_plugin", have("jax_cuda12_plugin"))
print("has_pjrt", have("jax_cuda12_pjrt"))
PY | tee /tmp/jax_backends.txt >/dev/null

if grep -q "has_plugin True" /tmp/jax_backends.txt; then HAS_PLUGIN=1; fi
if grep -q "has_pjrt True" /tmp/jax_backends.txt; then HAS_PJRT=1; fi

if [ "$HAS_PLUGIN" -eq 1 ] && [ "$HAS_PJRT" -eq 1 ]; then
  log "Both jax-cuda backends present → uninstall pjrt"
  PIP uninstall -y jax-cuda12-pjrt || true
  HAS_PJRT=0
fi

# Policy: keep LOCAL plugin (system CUDA) and remove pip NVIDIA runtime libs
if [ "$HAS_PLUGIN" -eq 0 ]; then
  log "Installing jax[cuda12-local] (PJRT CUDA plugin)"
  PIP install --no-cache-dir "jax[cuda12-local]>=0.4.26" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
fi

# Remove pip-provided NVIDIA CUDA stacks to avoid mixing with system CUDA
log "Removing pip NVIDIA CUDA stacks (if any) to prevent double-lib loads"
PIP uninstall -y \
  nvidia-cublas-cu12 nvidia-cuda-runtime-cu12 nvidia-cudnn-cu12 \
  nvidia-cuda-cupti-cu12 nvidia-cusolver-cu12 nvidia-cusparse-cu12 \
  nvidia-nvjitlink-cu12 nvidia-nvtx-cu12 nvidia-cufft-cu12 nvidia-curand-cu12 \
  || true

# Re-probe JAX devices
log "Probing JAX devices..."
$PY - <<'PY'
import sys
try:
    import jax
    print("[gpu-bootstrap] JAX", jax.__version__, "devices:", jax.devices())
    # sanity: ensure at most one CUDA PJRT present
    import importlib.util as u
    print("[gpu-bootstrap] has plugin:", u.find_spec("jax_cuda12_plugin") is not None)
    print("[gpu-bootstrap] has pjrt:", u.find_spec("jax_cuda12_pjrt") is not None)
except Exception as e:
    print("[gpu-bootstrap] JAX import/probe error:", e)
    sys.exit(1)
PY

log "Package snapshot:"
PIP_SHOW jax jaxlib jax-cuda12-plugin jax-cuda12-pjrt torch torchvision torchaudio || true

log "END"


Overwriting ../../.devcontainer/gpu_bootstrap.sh


In [18]:
%%writefile ../../.devcontainer/troubleshoot.sh
#!/bin/bash
set -euo pipefail

# DevContainer GPU Troubleshooting Script
# This script provides comprehensive diagnostics for GPU-enabled ML development environments

echo "🔍 DevContainer GPU Troubleshooting Report"
echo "=========================================="
echo "Generated: $(date)"
echo ""

# Function for logging
log() {
    echo "[$(date '+%Y-%m-%d %H:%M:%S')] $1"
}

# Function for section headers
section() {
    echo ""
    echo "📋 $1"
    echo "$(printf '%.0s-' {1..50})"
}

# Function for status indicators
status_ok() { echo "✅ $1"; }
status_warn() { echo "⚠️  $1"; }
status_error() { echo "❌ $1"; }
status_info() { echo "ℹ️  $1"; }

# Phase 1: System Prerequisites
section "System Prerequisites"

# Check if we're in a container
if [ -f /.dockerenv ]; then
    status_ok "Running in Docker container"
else
    status_warn "Not running in Docker container"
fi

# Check OS
if [ -f /etc/os-release ]; then
    . /etc/os-release
    status_info "OS: $PRETTY_NAME"
else
    status_warn "Could not determine OS"
fi

# Check Python environment
section "Python Environment"

PYTHON_VERSION=$(python --version 2>&1 || echo "Python not found")
status_info "Python: $PYTHON_VERSION"

if [ -n "${VIRTUAL_ENV:-}" ]; then
    status_ok "Virtual environment active: $VIRTUAL_ENV"
else
    status_warn "No virtual environment detected"
fi

# Check uv
if command -v uv >/dev/null 2>&1; then
    UV_VERSION=$(uv --version)
    status_ok "uv available: $UV_VERSION"
else
    status_error "uv not found"
fi

# Phase 2: GPU Hardware & Drivers
section "GPU Hardware & Drivers"

# Check for NVIDIA drivers
if command -v nvidia-smi >/dev/null 2>&1; then
    status_ok "nvidia-smi available"

    # Try to run nvidia-smi
    if nvidia-smi >/dev/null 2>&1; then
        status_ok "GPU accessible"

        # Get GPU info
        echo ""
        echo "GPU Information:"
        nvidia-smi --query-gpu=name,memory.total,driver_version,compute_cap --format=csv,noheader,nounits | while IFS=, read -r name memory driver compute; do
            echo "  GPU: $name"
            echo "  Memory: $memory MB"
            echo "  Driver: $driver"
            echo "  Compute Capability: $compute"
        done
    else
        status_error "nvidia-smi failed to run"
    fi
else
    status_error "nvidia-smi not available"
fi

# Check CUDA installation
section "CUDA Installation"

if [ -d "/usr/local/cuda" ]; then
    CUDA_VERSION=$(cat /usr/local/cuda/version.txt 2>/dev/null | grep "CUDA Version" | cut -d' ' -f3 || echo "Unknown")
    status_ok "CUDA installed: $CUDA_VERSION"

    # Check CUDA libraries
    if [ -f "/usr/local/cuda/lib64/libcudart.so" ]; then
        status_ok "CUDA runtime libraries found"
    else
        status_warn "CUDA runtime libraries not found"
    fi

    if [ -f "/usr/local/cuda/lib64/libcudnn.so" ]; then
        status_ok "cuDNN libraries found"
    else
        status_warn "cuDNN libraries not found"
    fi
else
    status_error "CUDA not installed"
fi

# Check environment variables
section "GPU Environment Variables"

ENV_VARS=(
    "NVIDIA_VISIBLE_DEVICES"
    "CUDA_VISIBLE_DEVICES"
    "LD_LIBRARY_PATH"
    "PATH"
    "XLA_PYTHON_CLIENT_PREALLOCATE"
    "XLA_PYTHON_CLIENT_MEM_FRACTION"
    "PYTORCH_CUDA_ALLOC_CONF"
    "TF_FORCE_GPU_ALLOW_GROWTH"
)

for var in "${ENV_VARS[@]}"; do
    if [ -n "${!var:-}" ]; then
        status_ok "$var: ${!var}"
    else
        status_warn "$var: not set"
    fi
done

# Phase 3: ML Framework Installation
section "ML Framework Installation"

# Check PyTorch
if python -c "import torch; print(f'PyTorch {torch.__version__}')" 2>/dev/null; then
    PYTORCH_VERSION=$(python -c "import torch; print(torch.__version__)" 2>/dev/null)
    status_ok "PyTorch installed: $PYTORCH_VERSION"

    # Check CUDA support
    if python -c "import torch; print('CUDA available:', torch.cuda.is_available())" 2>/dev/null; then
        CUDA_AVAILABLE=$(python -c "import torch; print(torch.cuda.is_available())" 2>/dev/null)
        if [ "$CUDA_AVAILABLE" = "True" ]; then
            status_ok "PyTorch CUDA support: Available"
            CUDA_COUNT=$(python -c "import torch; print(torch.cuda.device_count())" 2>/dev/null)
            status_info "CUDA devices: $CUDA_COUNT"
        else
            status_warn "PyTorch CUDA support: Not available"
        fi
    fi
else
    status_error "PyTorch not installed"
fi

# Check JAX
if python -c "import jax; print(f'JAX {jax.__version__}')" 2>/dev/null; then
    JAX_VERSION=$(python -c "import jax; print(jax.__version__)" 2>/dev/null)
    status_ok "JAX installed: $JAX_VERSION"

    # Check JAX devices
    if python -c "import jax; print('JAX devices:', jax.devices())" 2>/dev/null; then
        JAX_DEVICES=$(python -c "import jax; print(jax.devices())" 2>/dev/null)
        status_info "JAX devices: $JAX_DEVICES"

        # Check for GPU devices
        if echo "$JAX_DEVICES" | grep -q "gpu\|cuda"; then
            status_ok "JAX GPU support: Available"
        else
            status_warn "JAX GPU support: CPU only"
        fi
    fi
else
    status_error "JAX not installed"
fi

# TensorFlow removed for efficiency
status_info "TensorFlow: Removed for efficiency (PyTorch + JAX only)"

# Phase 4: Package Manager Status
section "Package Manager Status"

# Check uv sync status
if [ -f "pyproject.toml" ]; then
    status_ok "pyproject.toml found"

    if [ -f "uv.lock" ]; then
        status_ok "uv.lock found"
    else
        status_warn "uv.lock not found (run 'uv sync' to generate)"
    fi

    # Check if dependencies are installed
    if python -c "import pandas, numpy, matplotlib" 2>/dev/null; then
        status_ok "Core dependencies installed"
    else
        status_error "Core dependencies missing"
    fi
else
    status_warn "pyproject.toml not found in current directory"
fi

# Phase 5: Network & Connectivity
section "Network & Connectivity"

# Check internet connectivity
if ping -c 1 8.8.8.8 >/dev/null 2>&1; then
    status_ok "Internet connectivity available"
else
    status_error "No internet connectivity"
fi

# Check PyPI connectivity
if curl -s https://pypi.org/simple/ >/dev/null 2>&1; then
    status_ok "PyPI accessible"
else
    status_error "PyPI not accessible"
fi

# Phase 6: Recommendations
section "Recommendations"

echo "Based on the diagnostics above, here are some recommendations:"
echo ""

if ! command -v nvidia-smi >/dev/null 2>&1; then
    echo "❌ Install NVIDIA drivers and NVIDIA Container Toolkit"
    echo "   - For Ubuntu: sudo apt-get install nvidia-container-toolkit"
    echo "   - Restart Docker daemon after installation"
fi

if ! python -c "import torch; torch.cuda.is_available()" 2>/dev/null | grep -q "True"; then
    echo "⚠️  PyTorch CUDA support not working"
    echo "   - Check CUDA version compatibility"
    echo "   - Reinstall PyTorch with: uv pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128"
fi

if ! python -c "import jax; 'gpu' in str(jax.devices()).lower()" 2>/dev/null; then
    echo "⚠️  JAX GPU support not working"
    echo "   - Install JAX with CUDA: uv pip install 'jax[cuda12-local]' -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html"
fi

if [ ! -f "uv.lock" ]; then
    echo "⚠️  Run 'uv sync' to install dependencies"
fi

echo ""
echo "🔍 Troubleshooting completed at $(date)"





Overwriting ../../.devcontainer/troubleshoot.sh


In [19]:
%%writefile ../../.devcontainer/setup_env.sh
#!/usr/bin/env sh
set -eu
if [ ! -f /workspace/.env ]; then
  echo "📝  Generating default .env from template"
  cp /workspace/.devcontainer/.env.template /workspace/.env
fi



Overwriting ../../.devcontainer/setup_env.sh


In [20]:
%%writefile ../../.devcontainer/docker-compose.yml
# .devcontainer/docker-compose.yml - TensorFlow removed, optimized for faster builds
name: ${ENV_NAME:-cancer_bayes_iris_env}

services:
  datascience:
    build:
      context: ..
      dockerfile: .devcontainer/Dockerfile
      args:
        CUDA_TAG: ${CUDA_TAG:-12.8.0}
        PYTHON_VER: ${PYTHON_VER:-3.10}
        ENV_NAME: ${ENV_NAME:-cancer_bayes_iris_env}
      # Enable BuildKit for better caching
      target: ""
      cache_from:
        - nvidia/cuda:${CUDA_TAG:-12.8.0}-runtime-ubuntu22.04

    restart: unless-stopped
    depends_on:
      mlflow:
        condition: service_healthy

    # GPU configuration - simplified
    deploy:
      resources:
        reservations:
          devices:
            - driver: nvidia
              count: all
              capabilities: [gpu]

    init: true
    gpus: all
    shm_size: 4g  # Reduced from 8g since no TensorFlow
    ulimits:
      memlock: -1
      stack: 67108864

    environment:
      # Core configuration
      - PYTHON_VER=${PYTHON_VER:-3.10}
      - UV_PROJECT_ENVIRONMENT=/app/.venv

      # GPU Environment  
      - NVIDIA_VISIBLE_DEVICES=all
      - NVIDIA_DRIVER_CAPABILITIES=compute,utility
      - CUDA_VISIBLE_DEVICES=0
      - LD_LIBRARY_PATH=/app/.venv/lib:/usr/local/cuda/lib64

      # Simplified memory management (jemalloc)
      - LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libjemalloc.so.2
      - MALLOC_ARENA_MAX=1
      - MALLOC_TCACHE_MAX=0
      - PYTORCH_NO_CUDA_MEMORY_CACHING=1

      # JAX Configuration (RTX 5080 optimized)
      - XLA_PYTHON_CLIENT_PREALLOCATE=false
      - XLA_PYTHON_CLIENT_ALLOCATOR=platform
      - XLA_PYTHON_CLIENT_MEM_FRACTION=0.25
      - XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/local/cuda
      - JAX_PREALLOCATION_SIZE_LIMIT_BYTES=8589934592

      # PyTorch Configuration (RTX 5080 optimized)
      - PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512,expandable_segments:True

      # Jupyter
      - JUPYTER_TOKEN=${JUPYTER_TOKEN:-jupyter}

    volumes:
      - ..:/workspace:delegated
      - ../mlruns:/workspace/mlruns
      # Cache mounts for faster rebuilds
      - uv-cache:/root/.cache/uv

    ports:
      - "${HOST_JUPYTER_PORT:-8890}:8888"
      - "${HOST_TENSORBOARD_PORT:-6008}:6008"
      - "${HOST_EXPLAINER_PORT:-8050}:8050"
      - "${HOST_STREAMLIT_PORT:-8501}:8501"

    command: >
      bash -lc '
        echo "[boot] Activating environment...";
        source /app/.venv/bin/activate;
        echo "[boot] Starting Jupyter Lab...";
        jupyter lab --ip=0.0.0.0 --port=8888 --allow-root 
        --NotebookApp.token="${JUPYTER_TOKEN}" 
        --NotebookApp.allow_origin="*" 
        --NotebookApp.open_browser=false
      '

    healthcheck:
      test: ["CMD-SHELL", "python -c 'import jupyterlab' 2>/dev/null || exit 1"]
      interval: 30s
      timeout: 5s
      retries: 3
      start_period: 30s  # Reduced from 60s

    labels:
      - "com.docker.compose.project=${ENV_NAME:-cancer_bayes_iris_env}"
      - "com.docker.compose.service=datascience"
      - "description=AI/ML Dev Env (PyTorch+JAX GPU)"

  mlflow:
    image: ghcr.io/mlflow/mlflow:latest
    command: >
      mlflow server
      --host 0.0.0.0
      --port 5000
      --backend-store-uri sqlite:///mlflow.db
      --default-artifact-root /mlflow_artifacts
    environment:
      MLFLOW_EXPERIMENTS_DEFAULT_ARTIFACT_LOCATION: /mlflow_artifacts
    volumes:
      - ../mlruns:/mlflow_artifacts
      - ../mlflow_db:/mlflow_db
    ports:
      - "${HOST_MLFLOW_PORT:-5000}:5000"
    restart: unless-stopped
    healthcheck:
      test: ["CMD", "python", "-c", "import requests; requests.get('http://localhost:5000/health').raise_for_status()"]
      interval: 10s
      timeout: 3s
      retries: 5
      start_period: 15s  # Reduced from 30s

# Named volume for uv cache persistence
volumes:
  uv-cache:


Overwriting ../../.devcontainer/docker-compose.yml


In [21]:
%%writefile ../../.devcontainer/pyproject.toml
[project]
name = "cancer_bayes_iris_env"
version = "0.1.0"
description = "PyTorch and JAX GPU docker container (TensorFlow removed)"
authors = [{ name = "Geoffrey Hadfield" }]
license = "MIT"
readme = "README.md"
requires-python = ">=3.10,<3.13"

dependencies = [
  "pandas>=1.2.0",
  "numpy>=1.20.0",
  "matplotlib>=3.4.0",
  "mlflow>=2.10.2",
  "mlflow-skinny>=2.10.2",
  "scikit-learn>=1.4.2",
  "pymc>=5.0.0",
  "arviz>=0.14.0",
  "statsmodels>=0.13.0",
  "jupyterlab>=3.0.0",
  "seaborn>=0.11.0",
  "tabulate>=0.9.0",
  "shap>=0.40.0",
  "xgboost>=1.5.0",
  "lightgbm>=3.3.0",
  "catboost>=1.2.8,<1.3.0",
  "scipy>=1.7.0",
  "shapash[report]>=2.3.0",
  "shapiq>=0.1.0",
  "explainerdashboard==0.5.1",
  "ipywidgets>=8.0.0",
  "nutpie>=0.7.1",
  "numpyro>=0.18.0,<1.0.0",
  "pytensor>=2.18.3",
  "aesara>=2.9.4",
  "tqdm>=4.66.5",
  "pyarrow>=12.0.0",
  "optuna>=3.0.0",
  "optuna-integration[mlflow]>=0.2.0",
  "omegaconf>=2.3.0,<2.4.0",
  "hydra-core>=1.3.2,<1.4.0",
  "fastapi>=0.104.0",
  "uvicorn[standard]>=0.24.0",
  "pydantic>=2.0.0",
  "pydantic-settings",
  "kagglehub[pandas-datasets]>=0.3.12,<0.4.0",
  "duckdb~=1.3.2",
]

[project.optional-dependencies]
dev = [
  "pytest>=7.0.0",
  "black>=23.0.0",
  "isort>=5.0.0",
  "flake8>=5.0.0",
  "mypy>=1.0.0",
  "invoke>=2.2",
]

# GPU frameworks for Linux only (TensorFlow completely removed)
linux-gpu = [
  "torch      ; platform_system == 'Linux'",
  "torchvision; platform_system == 'Linux'",
  "torchaudio ; platform_system == 'Linux'",
  "jax[cuda12-local]>=0.4.26 ; platform_system == 'Linux'",
]

memory-management = [
  "psutil>=5.9.0",
  "memory-profiler>=0.61.0",
]

[tool.pytensor]
device = "cuda"
floatX = "float32"
allow_gc = true
optimizer = "fast_run"

[tool.uv]
compile-bytecode = true
link-mode = "copy"
python-downloads = "never"



Overwriting ../../.devcontainer/pyproject.toml


In [22]:
%%writefile ../../.devcontainer/tasks.py
# tasks.py  ── invoke ≥2.2
from invoke import task, Context  # type: ignore
from typing import List, Optional, Union

import os
import sys
import pathlib
import tempfile
import datetime as _dt
import atexit
import socket
import contextlib
import errno


BASE_ENV = pathlib.Path(__file__).parent


# Track temporary env files for cleanup
_saved_env_files: List[str] = []


def _parse_port(port: Union[str, int, None]) -> Optional[int]:
    """
    Parse and validate a port number.
    
    Args:
        port: Port number as string or int, or None
        
    Returns:
        Validated port number as int, or None if input was None
        
    Raises:
        ValueError: If port is invalid or out of range
    """
    if port is None:
        return None
        
    try:
        port_int = int(port)
        if not (0 < port_int < 65536):
            raise ValueError(f"Port {port_int} out of valid range (1-65535)")
        return port_int
    except (TypeError, ValueError) as e:
        raise ValueError(f"Invalid port value: {port}") from e


def _first_free_port(start: int = 5200) -> int:
    """Return the first TCP port >= *start* that is unused on localhost."""
    print(f"DEBUG: Searching for free port starting at {start}")  # Debug
    import socket
    import contextlib
    for port in range(start, 65535):
        with contextlib.closing(socket.socket()) as s:
            if s.connect_ex(("127.0.0.1", port)):
                print(f"DEBUG: Found free port {port}")  # Debug
                return port
    raise RuntimeError("No free port found")


def _free_port(start=5200) -> int:
    """Find a free port by letting the OS assign one."""
    print(f"DEBUG: Finding free port starting at {start}")  # Debug
    import socket
    import contextlib
    with contextlib.closing(
        socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    ) as s:
        s.bind(('', 0))
        port = s.getsockname()[1]
        print(f"DEBUG: Found free port {port}")  # Debug
        return port


def _port_free(host: str, port: int, timeout: float = 0.1) -> bool:
    """
    Return True iff *host:port* is NOT in use.

    Uses a non-blocking TCP connect – works on Linux, macOS, Windows,
    inside or outside WSL – and does **not** rely on lsof / netstat.
    """
    print(f"DEBUG: Checking if port {port} is free on {host}")  # Debug
    try:
        with contextlib.closing(
            socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        ) as s:
            s.settimeout(timeout)
            s.connect((host, port))
            print(f"DEBUG: Port {port} is in use")  # Debug
            return False      # connection succeeded ⇒ something listening
    except (OSError, socket.timeout):
        print(f"DEBUG: Port {port} is free")  # Debug
        return True           # connection failed ⇒ port is free


def _find_port(preferred: int, start: int = 5200) -> int:
    """
    Try to use preferred port, fall back to finding first available port.
    
    Args:
        preferred: The preferred port number to try first
        start: Where to start searching if preferred port is taken
        
    Returns:
        An available port number
    """
    print(f"DEBUG: Trying preferred port {preferred}")  # Debug
    if _port_free("127.0.0.1", preferred):
        return preferred
    return _first_free_port(start)


def _write_envfile(name: str, 
                   ports: Optional[dict[str, int]] = None) -> pathlib.Path:
    """
    Create a throw-away .env file for the current `invoke up` run.
    
    Docker-compose will use this to see the chosen host-ports. We include all
    services we know about; anything unset falls back to .env.template defaults.
    """
    env_lines = [f"ENV_NAME={name}"]
    mapping = {
        "jupyter": "HOST_JUPYTER_PORT",
        "tensorboard": "HOST_TENSORBOARD_PORT",
        "explainer": "HOST_EXPLAINER_PORT",
        "streamlit": "HOST_STREAMLIT_PORT",
        "mlflow": "HOST_MLFLOW_PORT",      # NEW
    }
    for svc, var in mapping.items():
        if ports and svc in ports:
            env_lines.append(f"{var}={ports[svc]}")
    env_lines.append(f"# generated {_dt.datetime.now().isoformat()}")
    tmp = tempfile.NamedTemporaryFile(
        "w", 
        delete=False, 
        prefix=".env.",
        dir=BASE_ENV
    )
    tmp.write("\n".join(env_lines))
    tmp.close()
    _saved_env_files.append(tmp.name)
    return pathlib.Path(tmp.name)


# Register cleanup function
def _cleanup_env_files() -> None:
    """Remove all temporary env files."""
    for path in _saved_env_files:
        try:
            os.remove(path)
        except OSError:
            pass


atexit.register(_cleanup_env_files)


def _compose(
    c: Context,
    cmd: str,
    name: str,
    rebuild: bool = False,
    force_pty: bool = False,
    ports: Optional[dict[str, int]] = None,
) -> None:
    """
    Wrapper around `docker compose` that also sanity-checks host ports.
    """
    # ---------- NEW pre-flight check --------------------------------------
    if ports:
        for svc, port in ports.items():
            if port is None:
                continue
            if not _port_free("127.0.0.1", int(port)):
                print(f"❌  Host port {port} already bound – "
                      f"{svc} cannot start. Choose another port (invoke up "
                      f"--{svc}-port XXXXX) or free it first.")
                sys.exit(1)

    env = {**os.environ, "ENV_NAME": name, "COMPOSE_PROJECT_NAME": name}
    
    # Add port overrides if provided
    if ports:
        port_mapping = {
            "jupyter": "HOST_JUPYTER_PORT",
            "tensorboard": "HOST_TENSORBOARD_PORT", 
            "explainer": "HOST_EXPLAINER_PORT",
            "streamlit": "HOST_STREAMLIT_PORT",
        }
        for service, port in ports.items():
            if service in port_mapping:
                env[port_mapping[service]] = str(port)
    
    use_pty = force_pty or (os.name != "nt" and sys.stdin.isatty())

    if not use_pty and not getattr(_compose, "_warned", False):
        print("ℹ️  PTY not supported – running without TTY.")
        _compose._warned = True  # type: ignore[attr-defined]

    if rebuild:
        full_cmd = f"docker compose -p {name} {cmd} --build"
    else:
        full_cmd = f"docker compose -p {name} {cmd}"
    c.run(full_cmd, env=env, pty=use_pty)


@task(
    help={
        "name": "Project/venv name (defaults to folder name)",
        "use_pty": "Force PTY even on non-POSIX hosts",
        "jupyter_port": "Jupyter Lab port (default: 8890)",
        "tensorboard_port": "TensorBoard port (default: auto-assigned)",
        "explainer_port": "Explainer Dashboard port (default: auto-assigned)", 
        "streamlit_port": "Streamlit port (default: auto-assigned)",
        "mlflow_port": "MLflow UI port (default: 5000, auto-assigns if busy)",
    }
)
def up(
    c,
    name: Optional[str] = None,
    rebuild: bool = False,
    detach: bool = True,
    use_pty: bool = False,
    jupyter_port: Union[str, int, None] = None,
    tensorboard_port: Union[str, int, None] = None,
    explainer_port: Union[str, int, None] = None,
    streamlit_port: Union[str, int, None] = None,
    mlflow_port: Union[str, int, None] = None,
) -> None:
    """Build (optionally --rebuild) & start the container with custom ports."""
    name = name or BASE_ENV.name

    # ---------- Parse and validate all ports -----------------
    try:
        jupyter_port = _parse_port(jupyter_port)
        tensorboard_port = _parse_port(tensorboard_port)
        explainer_port = _parse_port(explainer_port)
        streamlit_port = _parse_port(streamlit_port)
        mlflow_port = _parse_port(mlflow_port)
    except ValueError as e:
        print(f"❌ Port validation failed: {e}")
        sys.exit(1)

    # ---------- build dynamic port map -----------------
    ports = {}
    if jupyter_port is not None:
        ports["jupyter"] = jupyter_port
    if tensorboard_port is not None:
        ports["tensorboard"] = tensorboard_port
    if explainer_port is not None:
        ports["explainer"] = explainer_port
    if streamlit_port is not None:
        ports["streamlit"] = streamlit_port

    # ---------- Explainer auto-assign (NEW) ------------
    print("DEBUG: Starting explainer port assignment")  # Debug
    try:
        # Try to use the explainer's version first
        from src.mlops.explainer import _first_free_port  # type: ignore
        print("DEBUG: Successfully imported _first_free_port from explainer")  # Debug
    except ModuleNotFoundError:
        print("DEBUG: Failed to import _first_free_port, using local implementation")  # Debug
        # We'll use our local _first_free_port implementation
        pass

    if explainer_port is None:
        print("DEBUG: No explainer port specified, finding one")  # Debug
        explainer_port = _find_port(8050, 5200)
    elif not _port_free("127.0.0.1", explainer_port):
        print(f"DEBUG: Specified explainer port {explainer_port} is in use")  # Debug
        sys.exit(1)
    ports["explainer"] = explainer_port
    print(f"🔌 Explainer host-port → {explainer_port}")

    # ----- MLflow auto-assign (default 5000) -----------
    print("DEBUG: Starting MLflow port assignment")  # Debug
    if mlflow_port is None:
        print("DEBUG: No MLflow port specified, finding one")  # Debug
        mlflow_port = _find_port(5000, 5200)
    elif not _port_free("127.0.0.1", mlflow_port):
        print(f"DEBUG: Specified MLflow port {mlflow_port} is in use")  # Debug
        sys.exit(1)
    ports["mlflow"] = mlflow_port
    print(f"🔌 MLflow host-port → {mlflow_port}")

    # Generate environment file
    env_path = _write_envfile(name, ports)
    compose_cmd = "up -d" if detach else "up"

    _compose(
        c,
        f"--env-file {env_path} {compose_cmd}",
        name,
        rebuild=rebuild,
        force_pty=use_pty,
        ports=ports,
    )


@task(
    help={
        "name": "Project/venv name (defaults to folder name)",
    }
)
def stop(c, name: Optional[str] = None) -> None:
    """Stop and remove dev container (keeps volumes)."""
    name = name or BASE_ENV.name
    cmd = f"docker compose -p {name} down"
    try:
        c.run(cmd)
        print(f"\n🛑 Stopped and removed project '{name}'")
    except Exception:
        print(f"❌ No running containers found for project '{name}'")


@task
def shell(c, name: str | None = None) -> None:
    """Open an interactive shell inside the running container."""
    name = name or BASE_ENV.name
    cmd = f"docker compose -p {name} ps -q datascience"
    cid = c.run(cmd, hide=True).stdout.strip()
    c.run(f"docker exec -it {cid} bash", env={"ENV_NAME": name}, pty=False)


@task
def clean(c) -> None:
    """Prune stopped containers + dangling images."""
    c.run("docker system prune -f")


@task
def ports(c, name: str | None = None) -> None:
    """Show current port mappings for the named project."""
    name = name or BASE_ENV.name
    cmd = f"docker compose -p {name} ps --format table"
    try:
        c.run(cmd, hide=False)
        print(f"\n📊 Port mappings for project '{name}':")
        print("=" * 50)
    except Exception:
        print(f"❌ No running containers found for project '{name}'")
        print("\n💡 Usage examples:")
        print("  invoke up --name myproject --jupyter-port 8891")
        print("  invoke up --name myproject --jupyter-port 8892 \\")
        print("    --tensorboard-port 6009")


# --- utilities ---------------------------------------------------------------
def _norm(path: str | pathlib.Path) -> str:
    """Return a lower-case, forward-slash, no-trailing-slash version of *path*."""
    p = str(path).replace("\\", "/").rstrip("/").lower()
    return p

def _docker_projects_from_this_repo() -> set[str]:
    """
    Discover every Compose *project name* whose working_dir label ends with
    the current repo path.

    Works across Windows ↔ WSL ↔ macOS because we do suffix-match on a
    normalised path.
    """
    here_tail = _norm(pathlib.Path(__file__).parent.resolve())
    cmd = (
        "docker container ls -a "
        "--format '{{.Label \"com.docker.compose.project\"}} "
        "{{.Label \"com.docker.compose.project.working_dir\"}}' "
        "--filter label=com.docker.compose.project"
    )
    projects: set[str] = set()
    for line in os.popen(cmd).read().strip().splitlines():
        try:
            proj, wd = line.split(maxsplit=1)
        except ValueError:
            continue
        if _norm(wd).endswith(here_tail):
            projects.add(proj)
    return projects

# --- task --------------------------------------------------------------------
@task(
    help={
        "name": "Project name (defaults to folder). Ignored with --all.",
        "all":  "Remove *all* projects launched from this repo.",
        "rmi":  "Image-removal policy: all | local | none (default: local).",
    }
)
def down(c, name: str | None = None, all: bool = False, rmi: str = "local"):
    """
    Stop containers **and** fully delete every artefact so next `invoke up`
    starts from a clean slate.

    Examples
    --------
    invoke down                  # nuke current-folder project
    invoke down --name ml_project --rmi all   # wipe everything for ml_project
    invoke down --all            # tear down every project from this repo
    """
    if rmi not in {"all", "local", "none"}:
        raise ValueError("--rmi must be all | local | none")

    targets = _docker_projects_from_this_repo() if all else {name or BASE_ENV.name}
    flags = "-v --remove-orphans"
    if rmi != "none":
        flags += f" --rmi {rmi}"

    for proj in targets:
        try:
            c.run(f"docker compose -p {proj} down {flags}")
            print(f"🗑️  Removed project '{proj}'")
        except Exception:
            print(f"⚠️  Nothing to remove for '{proj}'")


@task(
    help={
        "yaml": "Path to dashboard.yaml file",
        "port": "Port to serve on (default: 8150)",
        "host": "Host to bind to (default: 0.0.0.0)",
    }
)
def dashboard(c, yaml: str, port: int = 8150, host: str = "0.0.0.0") -> None:
    """
    Serve a saved ExplainerDashboard from a YAML configuration file.
    
    This task allows you to re-serve dashboards that were previously saved
    with build_and_log_dashboard(save_yaml=True).
    
    Examples:
        invoke dashboard --yaml dashboard.yaml
        invoke dashboard --yaml dashboard.yaml --port 8200
    """
    import sys
    from pathlib import Path
    from src.mlops.explainer import load_dashboard_yaml
    
    yaml_path = Path(yaml)
    if not yaml_path.exists():
        print(f"❌ Dashboard YAML file not found: {yaml_path}")
        sys.exit(1)
    
    # Check if port is available
    if not _port_free(host, port):
        print(f"❌ Port {port} is already in use on {host}")
        sys.exit(1)
    
    try:
        print(f"🔄 Loading dashboard from {yaml_path}")
        dashboard_obj = load_dashboard_yaml(yaml_path)
        
        print(f"🌐 Serving ExplainerDashboard on {host}:{port}")
        dashboard_obj.run(port=port, host=host, use_waitress=True, open_browser=False)
        
    except Exception as e:
        print(f"❌ Failed to load or serve dashboard: {e}")
        sys.exit(1)



Overwriting ../../.devcontainer/tasks.py


In [23]:
%%writefile ../../tests/diagnose_devcontainer.py
#!/usr/bin/env python3
"""
Comprehensive diagnostic script for dev container issues.
Run this inside the container to diagnose Python environment and remote extension problems.
"""

import sys
import os
import subprocess
import json
from pathlib import Path


def run_command(cmd, description):
    """Run a command and return its output."""
    print(f"\n🔍 {description}")
    print("=" * 60)
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ {result.stdout.strip()}")
        else:
            print(f"❌ Error (code {result.returncode}): {result.stderr.strip()}")
        return result
    except Exception as e:
        print(f"❌ Exception: {e}")
        return None


def check_paths_and_environment():
    """Check Python paths and environment variables."""
    print("\n🐍 PYTHON ENVIRONMENT DIAGNOSTICS")
    print("=" * 60)
    
    # Python executable and version
    print(f"Python executable: {sys.executable}")
    print(f"Python version: {sys.version}")
    print(f"Python path: {sys.path[:3]}...")  # First few paths
    
    # Environment variables
    print(f"\nVIRTUAL_ENV: {os.environ.get('VIRTUAL_ENV', 'Not set')}")
    print(f"PATH (first 3): {':'.join(os.environ.get('PATH', '').split(':')[:3])}")
    
    # Virtual environment validation
    venv_path = Path('/app/.venv')
    if venv_path.exists():
        print(f"✅ Virtual environment exists at {venv_path}")
        print(f"   - bin directory: {list(venv_path.glob('bin/python*'))}")
        print(f"   - site-packages: {(venv_path / 'lib/python3.10/site-packages').exists()}")
    else:
        print(f"❌ Virtual environment NOT found at {venv_path}")


def check_key_packages():
    """Check if key packages are importable."""
    print("\n📦 PACKAGE IMPORT TESTS")
    print("=" * 60)
    
    packages = [
        'jax', 'torch', 'numpy', 'pandas', 'matplotlib', 
        'jupyterlab', 'streamlit', 'sklearn'
    ]
    
    for package in packages:
        try:
            if package == 'sklearn':
                import sklearn
                version = sklearn.__version__
            else:
                module = __import__(package)
                version = getattr(module, '__version__', 'unknown')
            print(f"✅ {package}: {version}")
        except ImportError as e:
            print(f"❌ {package}: Import failed - {e}")
        except Exception as e:
            print(f"⚠️  {package}: {e}")


def check_gpu_environment():
    """Check GPU-related environment variables."""
    print("\n🎮 GPU ENVIRONMENT VARIABLES")
    print("=" * 60)
    
    gpu_env_vars = [
        'XLA_PYTHON_CLIENT_PREALLOCATE',
        'XLA_PYTHON_CLIENT_ALLOCATOR', 
        'XLA_PYTHON_CLIENT_MEM_FRACTION',
        'JAX_PLATFORM_NAME',
        'XLA_FLAGS',
        'JAX_DISABLE_JIT',
        'JAX_ENABLE_X64',
        'JAX_PREALLOCATION_SIZE_LIMIT_BYTES',
        'TF_FORCE_GPU_ALLOW_GROWTH',
        'NVIDIA_VISIBLE_DEVICES',
        'NVIDIA_DRIVER_CAPABILITIES'
    ]
    
    for var in gpu_env_vars:
        value = os.environ.get(var, 'Not set')
        print(f"   {var}: {value}")


def check_gpu_support():
    """Check GPU support for JAX and PyTorch with enhanced diagnostics."""
    print("\n🎮 ENHANCED GPU SUPPORT CHECK")
    print("=" * 60)
    
    # JAX GPU check with detailed info
    try:
        import jax
        print(f"JAX version: {jax.__version__}")
        
        devices = jax.devices()
        print(f"JAX devices: {devices}")
        
        if devices:
            for i, device in enumerate(devices):
                print(f"   Device {i}: {device}")
                
        if any('gpu' in str(device).lower() or 'cuda' in str(device).lower() for device in devices):
            print("✅ JAX GPU/CUDA support detected!")
            
            # Test a simple computation
            try:
                import jax.numpy as jnp
                x = jnp.ones((1000, 1000))
                result = jnp.sum(x)
                print(f"   ✅ JAX GPU computation test passed: sum = {result}")
            except Exception as e:
                print(f"   ⚠️  JAX GPU computation test failed: {e}")
        else:
            print("⚠️  JAX GPU support not detected")
            print("   This might be due to GPU architecture compatibility")
            
    except Exception as e:
        print(f"❌ JAX GPU check failed: {e}")
    
    # PyTorch GPU check with enhanced info
    try:
        import torch
        print(f"\nPyTorch version: {torch.__version__}")
        print(f"PyTorch CUDA available: {torch.cuda.is_available()}")
        
        if torch.cuda.is_available():
            device_count = torch.cuda.device_count()
            print(f"✅ PyTorch CUDA device count: {device_count}")
            
            for i in range(device_count):
                try:
                    device_name = torch.cuda.get_device_name(i)
                    memory_total = torch.cuda.get_device_properties(i).total_memory
                    print(f"   Device {i}: {device_name}")
                    print(f"     Total memory: {memory_total / (1024**3):.1f} GB")
                except Exception as e:
                    print(f"   Device {i}: Error getting info - {e}")
            
            # Test a simple computation
            try:
                device = torch.device('cuda:0')
                x = torch.ones(1000, 1000, device=device)
                result = torch.sum(x)
                print(f"   ✅ PyTorch GPU computation test passed: sum = {result}")
            except Exception as e:
                print(f"   ⚠️  PyTorch GPU computation test failed: {e}")
        else:
            print("⚠️  PyTorch CUDA not available")
            print("   Check CUDA installation and GPU compatibility")
            
    except Exception as e:
        print(f"❌ PyTorch GPU check failed: {e}")


def check_workspace_mount():
    """Check if workspace is properly mounted."""
    print("\n📁 WORKSPACE MOUNT CHECK")
    print("=" * 60)
    
    workspace_path = Path('/workspace')
    if workspace_path.exists():
        print(f"✅ /workspace directory exists")
        try:
            contents = list(workspace_path.iterdir())[:10]  # First 10 items
            print(f"   Contents (first 10): {[p.name for p in contents]}")
            
            # Check for specific expected files
            expected_files = ['.devcontainer', 'pyproject.toml', 'docker-compose.yml']
            for file in expected_files:
                if (workspace_path / file).exists():
                    print(f"   ✅ Found: {file}")
                else:
                    print(f"   ❌ Missing: {file}")
        except Exception as e:
            print(f"   ❌ Error reading workspace: {e}")
    else:
        print(f"❌ /workspace directory does not exist")


def check_dev_container_config():
    """Check dev container configuration."""
    print("\n⚙️  DEV CONTAINER CONFIG CHECK")
    print("=" * 60)
    
    config_path = Path('/workspace/.devcontainer/devcontainer.json')
    if config_path.exists():
        print("✅ devcontainer.json found")
        try:
            with open(config_path) as f:
                config = json.load(f)
            print(f"   Name: {config.get('name', 'Not specified')}")
            print(f"   Python path: {config.get('customizations', {}).get('vscode', {}).get('settings', {}).get('python.defaultInterpreterPath', 'Not specified')}")
            print(f"   Workspace folder: {config.get('workspaceFolder', 'Not specified')}")
        except Exception as e:
            print(f"   ❌ Error reading config: {e}")
    else:
        print("❌ devcontainer.json not found")


def main():
    """Run all diagnostic checks."""
    print("🔍 DEV CONTAINER COMPREHENSIVE DIAGNOSTICS")
    print("=" * 80)
    print(f"Running from: {os.getcwd()}")
    print(f"User: {os.getenv('USER', 'unknown')}")
    print(f"Container hostname: {os.getenv('HOSTNAME', 'unknown')}")
    
    # System commands
    run_command("uv --version", "UV Version")
    run_command("which python", "Python Location")
    run_command("ls -la /app/.venv/", "Virtual Environment Contents")
    run_command("mount | grep workspace", "Workspace Mount Status")
    run_command("nvidia-smi", "NVIDIA GPU Status")
    
    # Python-based checks
    check_paths_and_environment()
    check_gpu_environment()
    check_key_packages()
    check_gpu_support()
    check_workspace_mount()
    check_dev_container_config()
    
    print("\n" + "=" * 80)
    print("🎯 SUMMARY & RECOMMENDATIONS")
    print("=" * 80)
    print("If you see issues:")
    print("1. ❌ Virtual env missing → Check Dockerfile uv sync step")
    print("2. ❌ Workspace not mounted → Check devcontainer.json mounts config")
    print("3. ❌ Packages missing → Check uv.lock and pip install steps")
    print("4. ⚠️  GPU not detected → Check docker-compose.yml gpu settings")
    print("5. 🔧 For VS Code issues → Check python.defaultInterpreterPath setting")
    print("6. 🎮 For GPU issues → Check NVIDIA drivers and CUDA compatibility")
    print("\n✅ All checks passed = ready for development!")


if __name__ == "__main__":
    main() 

Overwriting ../../tests/diagnose_devcontainer.py


In [24]:
%%writefile ../../tests/test_pytorch_jax_gpu.py
#!/usr/bin/env python3
"""
Test script to verify that PyTorch and JAX can access the GPU,
and that PyJAGS is working correctly.
"""

import sys


def test_pytorch_gpu():
    """Test PyTorch GPU availability and basic operations."""
    print("\n=== Testing PyTorch GPU ===")
    try:
        import torch
        print(f"PyTorch version: {torch.__version__}")
        print(f"CUDA available: {torch.cuda.is_available()}")
        
        if not torch.cuda.is_available():
            print("❌ PyTorch CUDA not available!")
            return False
        
        print(f"CUDA device count: {torch.cuda.device_count()}")
        print(f"Current device: {torch.cuda.current_device()}")
        print(f"Device name: {torch.cuda.get_device_name(0)}")
        
        # Run a simple test computation
        x = torch.rand(1000, 1000).cuda()
        y = torch.rand(1000, 1000).cuda()
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)
        
        start.record()
        z = torch.matmul(x, y)
        end.record()
        
        # Wait for GPU computation to finish
        torch.cuda.synchronize()
        print(f"Matrix multiplication time: {start.elapsed_time(end):.2f} ms")
        print(f"Result shape: {z.shape}")
        print("✅ PyTorch GPU test passed!")
        return True
    
    except ImportError:
        print("❌ PyTorch not found!")
        return False
    except Exception as e:
        print(f"❌ Error during PyTorch GPU test: {e}")
        return False


def test_jax_gpu():
    """Test JAX GPU availability and basic operations."""
    print("\n=== Testing JAX GPU ===")
    try:
        import jax, jax.numpy as jnp
        print(f"JAX version: {jax.__version__}")

        # Probe both: this won't crash if GPU plugin isn't present
        gpu_devs = []
        try:
            gpu_devs = jax.devices("gpu")
        except Exception:
            # Some builds don't register 'gpu' backend explicitly
            pass

        if not gpu_devs:
            # Fallback: inspect all devices for cuda/gpu strings
            devs = jax.devices()
            gpu_devs = [d for d in devs if "gpu" in str(d).lower() or "cuda" in str(d).lower()]

        if not gpu_devs:
            print("❌ No GPU devices found by JAX!")
            return False

        print(f"Available JAX GPU devices: {gpu_devs}")
        @jax.jit
        def matmul(a, b): return jnp.matmul(a, b)
        x = jnp.ones((1024, 1024))
        y = jnp.ones((1024, 1024))
        result = matmul(x, y)
        print(f"Result shape: {result.shape}")
        print("✅ JAX GPU test passed!")
        return True

    except ImportError:
        print("❌ JAX not found!")
        return False
    except Exception as e:
        print(f"❌ Error during JAX GPU test: {e}")
        return False



def test_pyjags():
    """Test PyJAGS installation and basic functionality."""
    print("\n=== Testing PyJAGS ===")
    try:
        import pyjags
        print(f"PyJAGS version: {pyjags.__version__}")
        
        # Create a simple model to verify that PyJAGS works
        code = """
        model {
            # Likelihood
            y ~ dnorm(mu, 1/sigma^2)
            
            # Priors
            mu ~ dnorm(0, 0.001)
            sigma ~ dunif(0, 100)
        }
        """
        
        # Sample data
        data = {'y': 0.5}
        
        # Initialize model with data
        model = pyjags.Model(code, data=data, chains=1, adapt=100)
        print("JAGS model initialized successfully!")
        
        # Sample from the model
        samples = model.sample(200, vars=['mu', 'sigma'])
        print("JAGS sampling completed successfully!")
        
        # Verify the samples
        mu_samples = samples['mu']
        sigma_samples = samples['sigma']
        print(f"mu mean: {mu_samples.mean():.4f}")
        print(f"sigma mean: {sigma_samples.mean():.4f}")
        
        print("✅ PyJAGS test passed!")
        return True
        
    except ImportError:
        print("❌ PyJAGS not found!")
        return False
    except Exception as e:
        print(f"❌ Error during PyJAGS test: {e}")
        return False


if __name__ == "__main__":
    print("Running GPU and PyJAGS verification tests...")
    
    pytorch_success = test_pytorch_gpu()
    jax_success = test_jax_gpu()
    pyjags_success = test_pyjags()
    
    print("\n=== Test Summary ===")
    print(f"PyTorch GPU: {'✅ PASS' if pytorch_success else '❌ FAIL'}")
    print(f"JAX GPU: {'✅ PASS' if jax_success else '❌ FAIL'}")
    print(f"PyJAGS: {'✅ PASS' if pyjags_success else '❌ FAIL'}")
    
    if pytorch_success and jax_success and pyjags_success:
        print("\n🎉 All tests passed! The container is working correctly.")
        sys.exit(0)
    else:
        print("\n❌ Some tests failed. Please check the output for details.")
        sys.exit(1)




Overwriting ../../tests/test_pytorch_jax_gpu.py


In [25]:
%%writefile ../../tests/enhanced_gpu_test_harness.py
#!/usr/bin/env python3
"""
Enhanced GPU Test Harness with Blackwell/SM_120 Support
======================================================

This module provides enhanced test functions for GPU frameworks with specific
support for NVIDIA Blackwell RTX 5080 and CUDA 12.8 compatibility.

Features:
- Enhanced JAX CUDA backend detection and conflict resolution
- TensorFlow INVALID_PTX error detection with diagnostic messages
- Comprehensive error reporting for Blackwell-specific issues

Usage:
    python tests/enhanced_gpu_test_harness.py
    
Or import individual functions:
    from tests.enhanced_gpu_test_harness import check_jax, check_tensorflow
"""

import sys
import gc
import textwrap


def check_jax():
    """
    Enhanced JAX GPU test with backend detection and Blackwell compatibility.
    
    This function provides:
    - Detection of CUDA backend plugins vs PJRT runtime
    - Identification of mixed backend conflicts
    - GPU device enumeration and computation testing
    - Clear error messages for common configuration issues
    """
    print("=== JAX ===")
    try:
        import jax, importlib.util as u
        print(f"jax.__version__={jax.__version__}")
        try:
            import jaxlib
            print(f"jaxlib.__version__={jaxlib.__version__}")
        except Exception as e:
            print("Could not import jaxlib:", repr(e))

        print("Backends: plugin?", u.find_spec("jax_cuda12_plugin") is not None,
              " pjrt?", u.find_spec("jax_cuda12_pjrt") is not None)

        try:
            devs = jax.devices()
            print(f"devices: {devs}")
            gpu = [d for d in devs if 'gpu' in str(d).lower() or 'cuda' in str(d).lower()]
            if not gpu:
                print("No JAX GPU devices found.")
            else:
                from jax import numpy as jnp
                x = jnp.ones((1024,1024), dtype=jnp.float32)
                y = (x @ x.T).sum()
                _ = y.block_until_ready()
                print("small matmul check (JAX GPU): OK")
        except RuntimeError as re:
            print("jax.devices() raised RuntimeError:", str(re))
        except Exception as e:
            print("jax.devices() failed:", repr(e))
    except Exception as e:
        print("JAX import failed:", repr(e))
    print("===========\n")


def check_tensorflow():
    """
    Enhanced TensorFlow GPU test with Blackwell INVALID_PTX detection.
    
    This function provides:
    - GPU device enumeration and memory growth configuration  
    - Explicit CUDA_ERROR_INVALID_PTX detection for Blackwell GPUs
    - Diagnostic messages for SM_120/CUDA 12.8 compatibility issues
    - Clear guidance for resolving PTX compilation failures
    """
    print("=== TensorFlow ===")
    try:
        import tensorflow as tf
        print(f"tf.__version__={tf.__version__}")
        try:
            gpus = tf.config.list_physical_devices("GPU")
            print(f"tf GPUs: {gpus}")
            if gpus:
                # Memory growth first
                for g in gpus:
                    try:
                        tf.config.experimental.set_memory_growth(g, True)
                    except Exception:
                        pass
                # Minimal compute test with explicit error surfacing
                try:
                    with tf.device("/GPU:0"):
                        a = tf.random.normal((1024,1024))
                        b = tf.random.normal((1024,1024))
                        c = tf.reduce_sum(tf.matmul(a, b))
                    _ = c.numpy()
                    print("small matmul check (TensorFlow GPU): OK")
                except Exception as e:
                    msg = str(e)
                    print("TensorFlow GPU op failed:", type(e).__name__, msg)
                    if "CUDA_ERROR_INVALID_PTX" in msg or "INVALID_PTX" in msg:
                        print(textwrap.dedent("""
>>> DIAG: INVALID_PTX on Blackwell usually means the TF wheel lacks SM_120 SASS and the PTX
          embedded in the wheel predates CUDA 12.8 support for Blackwell. Options:
          • install tf-nightly (post-CUDA-12.8 builds), or
          • use NVIDIA's TensorFlow NGC container built for CUDA 12.8+.
                        """).strip())
                    raise
            else:
                print("No TensorFlow GPU devices found.")
        except Exception as e:
            print("TensorFlow device query failed:", repr(e))
    except Exception as e:
        print("TensorFlow import failed:", repr(e))
    print("==================\n")


def check_pytorch():
    """
    Enhanced PyTorch GPU test with memory management.
    
    This function provides:
    - CUDA availability and device information
    - Memory cleanup after computation
    - Version and capability reporting
    """
    print("=== PyTorch ===")
    try:
        import torch
        print(f"torch.__version__={torch.__version__}")
        print(f"torch.version.cuda={torch.version.cuda}")
        print(f"torch.cuda.is_available()={torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"torch.cuda.device_count()={torch.cuda.device_count()}")
            print(f"current device name: {torch.cuda.get_device_name(0)}")
            
            # Test computation with memory cleanup
            x = torch.randn(1024, 1024, device="cuda")
            y = torch.randn(1024, 1024, device="cuda")
            z = (x @ y).sum().item()
            del x, y
            gc.collect()
            torch.cuda.empty_cache()
            print("small matmul check (PyTorch CUDA): OK")
        else:
            print("CUDA not available in PyTorch.")
    except Exception as e:
        print("PyTorch check failed:", repr(e))
    print("==============\n")


def comprehensive_gpu_test():
    """
    Run comprehensive GPU framework testing.
    
    This function tests all three major frameworks (PyTorch, JAX, TensorFlow)
    and provides a summary of results with specific guidance for failures.
    
    Returns:
        bool: True if all frameworks passed, False otherwise
    """
    print("🧪 COMPREHENSIVE GPU FRAMEWORK TEST")
    print("=" * 50)
    print("Testing GPU support for PyTorch, JAX, and TensorFlow...")
    print("Optimized for NVIDIA Blackwell RTX 5080 / CUDA 12.8")
    print("=" * 50)
    
    # Test all frameworks
    check_pytorch()
    check_jax()  
    check_tensorflow()
    
    print("=" * 50)
    print("✅ Comprehensive GPU test completed!")
    print("Check output above for any framework-specific issues.")
    print("=" * 50)
    
    return True


def main():
    """Main entry point for standalone execution."""
    import os
    
    # Show environment snapshot
    print("🔧 GPU ENVIRONMENT SNAPSHOT")
    print("=" * 30)
    env_vars = [
        "JAX_PLATFORM_NAME", "JAX_PLATFORMS", "CUDA_VISIBLE_DEVICES",
        "XLA_FLAGS", "NVIDIA_VISIBLE_DEVICES", "NVIDIA_DRIVER_CAPABILITIES",
        "PYTORCH_CUDA_ALLOC_CONF", "TF_FORCE_GPU_ALLOW_GROWTH"
    ]
    for var in env_vars:
        value = os.environ.get(var, "<unset>")
        print(f"{var}={value}")
    print("=" * 30)
    print()
    
    # Run comprehensive test
    comprehensive_gpu_test()


if __name__ == "__main__":
    main()







Overwriting ../../tests/enhanced_gpu_test_harness.py
